# Анализ данных интернет-магазина «В один клик»

## Описание проекта

Интернет-магазин «В один клик» продаёт разные товары: для детей, для дома, мелкую бытовую технику, косметику и даже продукты. Отчёт магазина за прошлый период показал, что активность покупателей начала снижаться. Привлекать новых клиентов уже не так эффективно: о магазине и так знает большая часть целевой аудитории. Возможный выход — удерживать активность постоянных клиентов. Сделать это можно с помощью персонализированных предложений.

«В один клик» — современная компания, поэтому её руководство не хочет принимать решения просто так — только на основе анализа данных и бизнес-моделирования. Она хочет разработать решение, которое позволит персонализировать предложения постоянным клиентам, чтобы увеличить их покупательскую активность.

Руководитель отдела уже сформировал подход к решению поставленной задачи:

1. Нужно промаркировать уровень финансовой активности постоянных покупателей. В компании принято выделять два уровня активности: «снизилась», если клиент стал покупать меньше товаров, и «прежний уровень».

2. Нужно собрать данные по клиентам по следующим группам:
- Признаки, которые описывают коммуникацию сотрудников компании с клиентом.
- Признаки, которые описывают продуктовое поведение покупателя. Например, какие товары покупает и как часто.
- Признаки, которые описывают покупательское поведение клиента. Например, сколько тратил в магазине.
- Признаки, которые описывают поведение покупателя на сайте. Например, как много страниц просматривает и сколько времени проводит на сайте.

3. Нужно построить модель, которая предскажет вероятность снижения покупательской активности клиента в следующие три месяца.

4. В исследование нужно включить дополнительные данные финансового департамента о прибыльности клиента: какой доход каждый покупатель приносил компании за последние три месяца.

5. Используя данные модели и данные о прибыльности клиентов, нужно выделить сегменты покупателей и разработать для них персонализированные предложения.

Осталось лишь реализовать сформулированный подход.

## Первоначальная настройка

### Автоматическое обновление и перезагрузка всех модулей

In [1]:
# автоматическое обновление всех модулей
%load_ext autoreload

# перезагрузка всех модулей
%autoreload 2

# обновление pip
!pip install --upgrade pip -q

# обновление всех нужных библиотек
!pip install --upgrade matplotlib numpy scikit-learn seaborn numba -q

ERROR: To modify pip, please run the following command:
C:\Users\tanae\.conda\envs\yp\python.exe -m pip install --upgrade pip -q


### Установка всех необходимых пакетов

In [2]:
# пакет missingno
!pip install missingno -q

# пакет phik
!pip install phik -q

# пакет deep-translator
!pip install deep-translator -q

# пакет optuna
!pip install optuna-integration -q

# пакет shap
!pip install shap -q

### Подключение всех необходимых библиотек

In [3]:
# стандартные библиотеки
import os
import re
import time

# библиотека numpy
import numpy as np

# библиотека pandas
import pandas as pd

# библиотека scipy
import scipy.stats as pt

# библиотека matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

# библиотека seaborn
import seaborn as sns

# библиотека sklearn
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import (OneHotEncoder, OrdinalEncoder, LabelEncoder,
                                   StandardScaler, MinMaxScaler, RobustScaler)
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import (confusion_matrix, accuracy_score, precision_score,
                             recall_score, precision_recall_curve, roc_auc_score)

# библиотека phik
import phik

# библлиотека missingno
import missingno as msno

# бибилотека optuna
import optuna
from optuna.integration import OptunaSearchCV

# библиотека shap
import shap

# другие библиотеки
from IPython.display import Markdown
from deep_translator import GoogleTranslator

C:\Users\tanae\.conda\envs\yp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Задание всех необходимых опций

In [4]:
# настройка pandas для отображения всех столбцов
pd.set_option('display.max_columns', None)

# установка палитры
sns.set_palette('muted')

# создание константы RANDOM_STATE
# для воспроизводимости результатов
RANDOM_STATE = 42

### Функции для загрузки данных

#### `create_dataframe()`

In [5]:
def create_dataframe(pth1, pth2, sep=',', decimal='.'):
    '''
    Аргументы функции: 
    - pth1: путь к датасету (строка),
    - pth2: альтернативный путь
      к датасету (строка),
    - sep: разделитель столбцов (строка),
    - decimal: десятичный разделитель (строка).
    
    Функция возвращает датафрейм по заданным путям pth1
    и pth2 к датасету.
    '''
    if os.path.exists(pth1):      
        return pd.read_csv(pth1, sep=sep, decimal=decimal)
    elif os.path.exists(pth2):
        return pd.read_csv(pth2, sep=sep, decimal=decimal)
    else:
        print('Ошибка чтения')

#### `data_review()`

In [6]:
def data_review(df):
    '''
    Аргументы функции:
    - df: заданный датафрейм.
    
    Функция выводит информацию
    о заданном датафрейме df.
    '''
    # вывод первых пяти строк данных
    display(Markdown('##### Первые 5 строк датасета'))
    display(df.head())
    
    # вывод информации о столбцах и типах данных
    display(Markdown('##### Информация о столбцах и типах данных'))
    print(df.info())
    
    # вывод статистического описания числовых столбцов
    display(Markdown('##### Статистическое описание числовых столбцов'))
    display(df.describe())
    
    # вывод кол-ва уникальных значений столбцов типа object
    display(Markdown('##### Кол-во уникальных значений столбцов типа object'))
    print(df.select_dtypes(include=['object']).nunique())

### Функции для предобработки данных

#### `rename_columns()`

In [7]:
def rename_columns(df, ending='', col_no_ending=[], inf=True):
    '''
    Аргументы функции:
    - df: заданный датафрейм,
    - ending: определяет окончание названий
    всех столбцов, кроме col_no_ending (строка),
    - col_no_ending: определяет столбцы без окончания
    (список),
    - inf: определяет выводить ли информацию
    о столбцах (True или False).
    
    Функция переименовывает названия столбцов
    заданного датафрейма df согласно PEP8.
    '''
    # создание объекта Translator
    translator = GoogleTranslator(source='auto',
                                  target='en')
    
    # создание списка для хранения
    # новых назвний столбцов
    new_columns = []
    
    # запуск цикла по названиям столбцов
    for column in df.columns:
        
        # перевод названия столбца
        name = translator.translate(column)
        
        # перевод названия столбца
        # в нижний регистр
        name = name.lower()
        
        # замена пробелов и других
        # разделителей на подчеркивания
        name = re.sub(r'\s+', '_', name)
        
        # добавление окончания
        if not column in col_no_ending:
            name = name + ending
        
        # вывод информации о столбцах
        if inf == True:
            print(column, '-', name)
        
        # сохранение нового названия столбца
        new_columns.append(name)
    
    # переименование столбцов
    df.columns = new_columns
    
    # возвращение заданного датафрейма
    return df

#### `change_data_types()`

In [8]:
def change_data_types(df, column_types):
    '''
    Аргументы функции:
    - df: выбранный датафрейм,
    - column_types: заданный словарь
      (ключ - название столбца,
       значение - нужный тип данных).
                       
    Функция изменяет типы данных столбцов выбранного
    датафрейма df согласно заданному словарю column_types.
    '''
    # изменение типов данных столбцов
    for column, col_type in column_types.items():
        try:
            df[column] = df[column].astype(col_type)
        except ValueError:
            print(f'Ошибка преобразования столбца "{column}" в тип {col_type}')
        except KeyError:
            print(f'Столбец "{column}" отсутствует в DataFrame')
    
    # проверка
    print(df.dtypes)
    
    # возвращение заданного датафрейма
    return df

#### `obvious_duplicates()`

In [9]:
def obvious_duplicates(df, comb_cols=[]):
    '''
    Аргументы функции:
    - df: заданный датафрейм,
    - comb_cols: комбинации столбцов,
      по которым нужно проверить наличие
      явных дубликатов (двумерный список).
    
    Функция выводит количество явных дубликатов.
    '''
    # вывод столбцов и кол-ва дубликатов,
    # находящихся в этих столбцах
    for column in df.columns:
        print(f'{column}: {df.duplicated(subset=column).sum()}')
    
    # отступ
    if comb_cols != []:
        print()
    
    # кол-во дубликатов по комбинациям столбцов
    for column in comb_cols:
        print(f'{column}: {df.duplicated(subset=column).sum()}')
    
    # вывод кол-ва явных дубликатов всего датафрейма
    print(f'\nкол-во абсолютных явных дубликатов: {df.duplicated().sum()}')

#### `object_unique()`

In [10]:
def object_unique(df):
    '''
    Аргументы функции:
    - df: заданный датафрейм.
    
    Функция выводит все уникальные значения
    по столбцам типа object заданного датафрейма df.
    '''
    # запуск цикла по столбцам датафрейма
    for column in df.columns:
        # если строчный тип данных, то 
        if df[column].dtypes == 'object':
            # вывод уникальных значений
            print(f'{column}: {df[column].unique()}')

#### `more_describe()`

In [11]:
def more_describe(df, column, hue=None):
    '''
    Аргументы функции:
    - df: выбранный датафрейм,
    - column: заданный числовой столбец (строка),
    - hue: дополнительная разбивка (строка).
    
    Функция выводит расширенное статистическое описание
    заданного числового столбца выбранного датафрейма.
    '''
    if hue != None:
        
        description = pd.DataFrame(columns=df[hue].unique())
        
        # перебор по категориям
        for category in df[hue].unique():
            
            # фильтрация данных по категории
            category_data = df[df[hue] == category]
            
            # получение стандартного описания
            description[category] = category_data[column].describe()
            
            # вычисление квартилей и межквартильного размаха
            # для каждой категории
            q1 = category_data[column].quantile(q=0.25)
            q3 = category_data[column].quantile(q=0.75)
            iqr = q3 - q1
            
            # определение выбросов
            ems = ((category_data[column] < (q1 - 1.5 * iqr)) | 
                   (category_data[column] > (q3 + 1.5 * iqr)))
            
            # подсчет выбросов и их характеристик
            count = category_data[ems].shape[0]
            share = count / category_data[column].count() * 100
            min_em = category_data[ems][column].min()
            max_em = category_data[ems][column].max()
            
            # подготовка результатов
            description.loc['iqr', category] = iqr
            description.loc['lower_bound', category] = q1
            description.loc['upper_bound', category] = q3
            description.loc['outliers_count', category] = count
            description.loc['outliers_pct', category] = share
            description.loc['outlier_min', category] = min_em
            description.loc['outlier_max', category] = max_em
    
    else:
        
        # получение стандартного описания
        description = df[column].describe()
        
        # вычисление квартилей и межквартильного размаха
        q1 = df[column].quantile(q=0.25)
        q3 = df[column].quantile(q=0.75)
        iqr = q3 - q1
        
        # определение выбросов
        ems = ((df[column] < (q1-1.5*iqr)) |\
               (df[column] > (q3+1.5*iqr)))
        
        # подсчет выбросов и их характеристик
        count = df[ems][column].count()
        share = round((count / df[column].count() * 100), 2)
        min_em = round(df[ems][column].min(), 2)
        max_em = round(df[ems][column].max(), 2)
        
        # подготовка результатов
        description['iqr'] = iqr
        description['lower_bound'] = round(q1, 2)
        description['upper_bound'] = round(q3, 2)
        description['outliers_count'] = count
        description['outliers_percentage'] = share
        description['outliers_min'] = min_em
        description['outliers_max'] = max_em
    
    return description

### Функции для визуализации

#### `num_plots()`

In [12]:
def num_plots(df, x, hue=None, bins=0, xlim=(),
              xlabel='', ylabel='', legend_title='',
              title='', figsize=(16, 7)):
    '''
    Аргументы функции:
    - df: заданный датафрейм,
    - x: значения по оси X (строка),
    - hue: дополнительная разбивка (строка),
    - bins: количество корзин (целое число),
    - xlim: диапазон значений по оси X (кортеж),
    - xlabel: название оси X (строка),
    - ylabel: название оси Y (строка),
    - legend_title: название легенды (строка),
    - title: название рисунка (строка),
    - figsize: размер рисунка (кортеж).
    
    Функция составляет гистограмму (столбчатую диаграмму)
    и ящик с усами по заданному датафрейму.
    '''
    # создание подграфиков
    fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2,
                                   figsize=figsize)
    
    # создание гистограммы или столбчатой диаграммы
    if bins == 0:
        sns.countplot(data=df, x=x, hue=hue, ax=ax1)
        for i in range(0, df[hue].nunique()):
            ax1.bar_label(ax1.containers[i], padding=2)
        ax1.set_title('Столб. диаграмма', pad=10, fontsize=14)
    else:
        sns.histplot(data=df, x=x, hue=hue,
                     bins=bins, kde=True, ax=ax1)
        ax1.grid()
        ax1.set_title('Гистограмма', pad=10, fontsize=14)
    if xlim != ():
        ax1.set_xlim(xlim[0], xlim[1])
    ax1.set_xlabel(xlabel, labelpad=13, fontsize=12)
    ax1.set_ylabel(ylabel, labelpad=13, fontsize=12)
    if hue is not None:
        legend = ax1.get_legend()
        legend.set_title(legend_title)
    
    # создание ящика с усами
    sns.boxplot(data=df, x=x, y=hue, ax=ax2)
    ax2.set_title('Ящик с усами', pad=10, fontsize=14)
    ax2.set_xlabel(xlabel, labelpad=13, fontsize=12)
    ax2.set_ylabel(legend_title.lower(), labelpad=13, fontsize=12)
    ax2.tick_params(axis='y', rotation=45)
    ax2.grid()
    
    # создание названия рисунка
    fig.suptitle(title, fontsize=18)
    
    # отображение графиков
    plt.tight_layout(rect=[0, 0, 1, 0.98])
    plt.show()

#### `cat_plots()`

In [13]:
def cat_plots(df, column, hue, legend_title='',
              title='', figsize=(16, 7)):
    '''
    Аргументы функции:
    - df: заданный датафрейм,
    - column: столбец с категор. значениями (строка),
    - hue: дополнительная разбивка (строка),
    - legend_title: название легенды (строка),
    - title: название рисунка (строка),
    - figsize: размер рисунка (кортеж).
    
    Функция составляет круговую диаграмму
    по заданному датафрейму.
    '''
    # функция для подписей круговой диаграммы
    def captions(pct, allvalues):
        absolute = round(pct / 100.*np.sum(allvalues))
        return '{:.1f}%\n({:d})'.format(pct, absolute)
    
    # кол-во уникальных значений
    categories = df[hue].unique()
    count_unique1 = df[df[hue] == categories[0]][column].value_counts()
    count_unique2 = df[df[hue] == categories[1]][column].value_counts()
    
    # задание равномерных отступов
    explode1 = [0.05] * len(count_unique1)
    explode2 = [0.05] * len(count_unique2)
    
    # создание подграфиков
    fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2,
                                   figsize=figsize)
    
    # создание первой круговой диаграммы
    wedges1, texts1, autotexts1 = ax1.pie(count_unique1,
                                          autopct=lambda pct: captions(pct, count_unique1),
                                          explode=explode1)
    ax1.legend(wedges1, count_unique1.index.tolist(),
               title=legend_title, loc='center left',
               bbox_to_anchor=(1, 0.5))
    plt.setp(autotexts1, size=12)
    ax1.set_title(categories[0], pad=0, fontsize=14)
    
    # создание второй круговой диаграммы
    wedges2, texts2, autotexts2 = ax2.pie(count_unique2,
                                          autopct=lambda pct: captions(pct, count_unique2),
                                          explode=explode2)
    ax2.legend(wedges2, count_unique2.index.tolist(),
               title=legend_title, loc='center left',
               bbox_to_anchor=(1, 0.5))
    plt.setp(autotexts2, size=12)
    ax2.set_title(categories[1], pad=0, fontsize=14)
    
    # создание названия рисунка
    fig.suptitle(title, fontsize=18)
    
    # отображение графика
    plt.tight_layout(rect=[0, 0, 1, 0.98])
    plt.show()

#### `piechart()`

In [14]:
def piechart(series1, series2=None,
             title1='', title2='',
             legend_title='', title='',
             figsize=(10, 8)):
    '''
    Аргументы функции:
    - series1: 1-ый заданный объект Series,
    - series2: 2-ой заданный объект Series,
    - title1: название первой круговой диаграммы (строка),
    - title2: название второй круговой диаграммы (строка),
    - legend_title: название легенды (строка),
    - title: название рисунка (строка),
    - figsize: размер графика (кортеж).
    
    Функция составляет круговые диаграммы
    по заданным объектам Series.
    '''
    # функция для подписей круговой диаграммы
    def captions(pct, allvalues):
        absolute = round(pct / 100.*np.sum(allvalues))
        return '{:.1f}%\n({:d})'.format(pct, absolute)
    
    if series2 is not None:
        
        # кол-во уникальных значений
        count_unique1 = series1.value_counts()
        count_unique2 = series2.value_counts()
        
        # задание равномерных отступов
        explode1 = [0.05] * len(count_unique1)
        explode2 = [0.05] * len(count_unique2)
        
        # создание подграфиков
        fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2,
                                       figsize=figsize)
        
        # создание первой круговой диаграммы
        wedges1, texts1, autotexts1 = ax1.pie(count_unique1,
                                              autopct=lambda pct: captions(pct, count_unique1),
                                              explode=explode1)
        ax1.legend(wedges1, count_unique1.index.tolist(),
                   title=legend_title, loc='center left',
                   bbox_to_anchor=(1, 0.5))
        plt.setp(autotexts1, size=12)
        ax1.set_title(title1, pad=0, fontsize=14)
        
        # создание второй круговой диаграммы
        wedges2, texts2, autotexts2 = ax2.pie(count_unique2,
                                              autopct=lambda pct: captions(pct, count_unique2),
                                              explode=explode2)
        ax2.legend(wedges2, count_unique2.index.tolist(),
                   title=legend_title, loc='center left',
                   bbox_to_anchor=(1, 0.5))
        plt.setp(autotexts2, size=12)
        ax2.set_title(title2, pad=0, fontsize=14)
        
        # создание названия рисунка
        fig.suptitle(title, fontsize=18)
        
        # настройки параметров расположения элементов на графике
        plt.tight_layout(rect=[0, 0, 1, 0.98])
        
    else:
        
        # кол-во уникальных значений
        count_unique = series1.value_counts()
        
        # задание равномерных отступов
        explode = [0.05] * len(count_unique)
    
        # задание размеров диаграммы
        fig, ax = plt.subplots(figsize=figsize)
        
        # создание круговой диаграммы
        wedges, texts, autotexts = plt.pie(count_unique,
                                           autopct=lambda pct: captions(pct, count_unique),
                                           explode=explode)
        
        # создание названия графика
        fig.suptitle(title, fontsize=16, y=0.87, va='top')
        
        # создание легенды
        ax.legend(wedges, count_unique.index.tolist(),
                  title=legend_title,
                  loc='center left',
                  bbox_to_anchor=(1, 0.5))
        
        # задание размеров подписей
        # plt.setp(autotexts, size=12)
        
    # отображение графика
    plt.show()

#### `plotbar()`

In [15]:
def plotbar(df1, x, y, df2=None, hue=None, order=None,
            xlabel='', ylabel='', legend_title='',
            title1='', title2='', title='', figsize=(16, 7)):
    '''
    Аргументы функции: 
    - df1: 1-ый заданный датафрейм,
    - x: значения по оси X (строка),
    - y: значения по оси Y (строка),
    - df2: 2-ой заданный датафрейм,
    - hue: дополнительная разбивка (строка),
    - order: порядок значений для осей (список),
    - xlabel: название оси X (строка),
    - ylabel: название оси Y (строка),
    - legend_title: название легенды (строка),
    - title1: название 1-ой столбчатой диаграммы (строка),
    - title2: название 2-ой столбчатой диаграммы (строка),
    - title: название рисунка (строка),
    - figsize: размер графика (кортеж).
    
    Функция составляет столбчатые диаграммы
    по заданным датафреймам.
    '''
    if df2 is not None:
        
        # создание подграфиков
        fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2,
                                       figsize=figsize)
        
        # создание 1-ой столбчатой диаграммы
        sns.barplot(data=df1, x=x, y=y, hue=hue, order=order,
                    errorbar=None, ax=ax1)
        ax1.set_title(title1, pad=10, fontsize=14)
        ax1.set_xlabel(xlabel, labelpad=13, fontsize=12)
        ax1.set_ylabel(ylabel, labelpad=13, fontsize=12)
        if hue is not None:
            legend1 = ax1.legend(title=legend_title,
                                 loc='center left',
                                 bbox_to_anchor=(1, 0.5))
        
        # создание 2-ой столбчатой диаграммы
        sns.barplot(data=df2, x=x, y=y, hue=hue, order=order,
                    errorbar=None, ax=ax2)
        ax2.set_title(title2, pad=10, fontsize=14)
        ax2.set_xlabel(xlabel, labelpad=13, fontsize=12)
        ax2.set_ylabel(ylabel, labelpad=13, fontsize=12)
        if hue is not None:
            legend2 = ax2.legend(title=legend_title,
                                 loc='center left',
                                 bbox_to_anchor=(1, 0.5))
        
        # создание названия рисунка
        fig.suptitle(title, fontsize=18)
        
        # настройки параметров расположения элементов на графике
        plt.tight_layout(rect=[0, 0, 1, 0.98])
        
    else:
        
        # задание размеров графика
        fig, ax = plt.subplots(figsize=figsize)
        
        # создание столбчатой диаграммы
        sns.barplot(data=df1, x=x, y=y, hue=hue, order=order,
                    errorbar=None, ax=ax)
        
        # создание названия графика
        plt.title(title, pad=10, fontsize=16)
        
        # создание подписей над осями графика
        plt.xlabel(xlabel, labelpad=15, fontsize=12)
        plt.ylabel(ylabel, labelpad=15, fontsize=12)
        
        # создание легенды
        if hue is not None:
            legend = ax.legend(title=legend_title,
                               loc='center left',
                               bbox_to_anchor=(1, 0.5))
    
    # отображение графиков
    plt.show()

#### `corr_heatmap()`

In [16]:
def corr_heatmap(df, interval_cols, title='',
                 figsize=(15, 10)):
    '''
    Аргументы функции: 
    - df: заданный датафрейм,
    - interval_cols: столбцы с непрерывными
    значениями (список),
    - title: название корр. тепловой карты (строка),
    - figsize: размер графика (кортеж).
    
    Функция составляет корреляционную тепловую карту
    по заданному датафрейму на основе коэффициента phik.
    '''
    # задание размеров графика
    fig, ax = plt.subplots(figsize=figsize)
    
    # создание матрицы корреляций phik
    matrix_corr = df.phik_matrix(interval_cols=interval_cols,
                                 verbose=False)
    
    # создание тепловой карты
    sns.heatmap(matrix_corr, annot=True, fmt='.1g', vmin=0,
                vmax=1, center=0.5, ax=ax)
    
    # поворот меток
    plt.xticks(rotation=90)
    plt.yticks(rotation=0)
    
    # создание названия графика
    plt.title(title, pad=13, fontsize=16)
    
    # отображение графика
    plt.show()

#### `conf_matrix()`

In [17]:
def conf_matrix(y_true, y_pred, le=None,
                title='', figsize=(13, 9)):
    '''
    Аргументы функции: 
    - y_true: истинные значения,
    - y_pred: предсказанные значения,
    - le: кодировщик целевого признака LabelEncoder,
    - title: название матрицы ошибок (строка),
    - figsize: размер графика (кортеж).
    
    Функция составляет матрицу ошибок по заданным истинным
    и предсказанным значениям.
    '''
    # подсчет матрицы ошибок
    cm = confusion_matrix(y_true, y_pred)
    
    # задание размеров графика
    fig, ax = plt.subplots(figsize=figsize)
    
    # создание тепловой карты
    sns.heatmap(cm, annot=True, fmt='d', ax=ax)
    
    # метки
    if le is not None:
        ax.set_xticklabels(le.classes_, rotation=0)
        ax.set_yticklabels(le.classes_, rotation=90)
    
    # подпись осей
    plt.ylabel('истина',  labelpad=15, fontsize=12)
    plt.xlabel('прогнозы',  labelpad=15, fontsize=12)
    
    # создание названия графика
    plt.title(title, pad=13, fontsize=16)
    
    # отображение графика
    plt.show()

#### `scatter_plot()`

In [18]:
def scatter_plot(df, x, y, hue=None, xlabel='', ylabel='',
                 legend_title='', title='', figsize=(15, 10)):
    '''
    Аргументы функции:
    - df: заданный датафрейм,
    - x: значения по оси X (строка),
    - y: значения по оси Y (строка),
    - hue: дополнительная разбивка (строка),
    - xlabel: название оси X (строка),
    - ylabel: название оси Y (строка),
    - legend_title: название легенды (строка),
    - title: название графика (строка),
    - figsize: размер графика (кортеж).
    
    Функция составляет диаграмму рассеяния по заданным
    столбцам выбранного датафрейма.
    '''
    # задание размеров графика
    fig, ax = plt.subplots(figsize=figsize)
    
    # создание диаграммы рассеяния
    sns.scatterplot(data=df, x=x, y=y, hue=hue, ax=ax)
    
    # создание подписей над осями графика
    plt.xlabel(xlabel, labelpad=15, fontsize=12)
    plt.ylabel(ylabel, labelpad=15, fontsize=12)
    
    # создание названия легенды
    if hue != None:
        ax.legend(title=legend_title)
    
    # создание названия графика
    plt.title(title, pad=10, fontsize=16)
    
    # создание сетки
    plt.grid()
    
    # отображение графика
    plt.show()

## Загрузка данных

### Датасет `market_file.csv`

In [19]:
# создание датафрейма
market_file = create_dataframe('datasets/market_file.csv',
                               '/datasets/market_file.csv')

# обзор данных
data_review(market_file)

##### Первые 5 строк датасета

,id,Покупательская активность,Тип сервиса,Разрешить сообщать,Маркет_актив_6_мес,Маркет_актив_тек_мес,Длительность,Акционные_покупки,Популярная_категория,Средний_просмотр_категорий_за_визит,Неоплаченные_продукты_штук_квартал,Ошибка_сервиса,Страниц_за_визит
0,215348,Снизилась,премиум,да,3.4,5,121,0.00,Товары для детей,6,2,1,5
1,215349,Снизилась,премиум,да,4.4,4,819,0.75,Товары для детей,4,4,2,5
2,215350,Снизилась,стандартт,нет,4.9,3,539,0.14,Домашний текстиль,5,2,1,5
3,215351,Снизилась,стандартт,да,3.2,5,896,0.99,Товары для детей,5,0,6,4
4,215352,Снизилась,стандартт,нет,5.1,3,1064,0.94,Товары для детей,3,2,3,2


##### Информация о столбцах и типах данных

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1300 entries, 0 to 1299
Data columns (total 13 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   id                                   1300 non-null   int64  
 1   Покупательская активность            1300 non-null   object 
 2   Тип сервиса                          1300 non-null   object 
 3   Разрешить сообщать                   1300 non-null   object 
 4   Маркет_актив_6_мес                   1300 non-null   float64
 5   Маркет_актив_тек_мес                 1300 non-null   int64  
 6   Длительность                         1300 non-null   int64  
 7   Акционные_покупки                    1300 non-null   float64
 8   Популярная_категория                 1300 non-null   object 
 9   Средний_просмотр_категорий_за_визит  1300 non-null   int64  
 10  Неоплаченные_продукты_штук_квартал   1300 non-null   int64  
 11  Ошибка_сервиса                

##### Статистическое описание числовых столбцов

,id,Маркет_актив_6_мес,Маркет_актив_тек_мес,Длительность,Акционные_покупки,Средний_просмотр_категорий_за_визит,Неоплаченные_продукты_штук_квартал,Ошибка_сервиса,Страниц_за_визит
count,1300.000000,1300.000000,1300.000000,1300.000000,1300.000000,1300.00000,1300.000000,1300.000000,1300.000000
mean,215997.500000,4.253769,4.011538,601.898462,0.319808,3.27000,2.840000,4.185385,8.176923
std,375.421985,1.014814,0.696868,249.856289,0.249843,1.35535,1.971451,1.955298,3.978126
min,215348.000000,0.900000,3.000000,110.000000,0.000000,1.00000,0.000000,0.000000,1.000000
25%,215672.750000,3.700000,4.000000,405.500000,0.170000,2.00000,1.000000,3.000000,5.000000
50%,215997.500000,4.200000,4.000000,606.000000,0.240000,3.00000,3.000000,4.000000,8.000000
75%,216322.250000,4.900000,4.000000,806.000000,0.300000,4.00000,4.000000,6.000000,11.000000
max,216647.000000,6.600000,5.000000,1079.000000,0.990000,6.00000,10.000000,9.000000,20.000000


##### Кол-во уникальных значений столбцов типа object

Покупательская активность    2
Тип сервиса                  3
Разрешить сообщать           2
Популярная_категория         6
dtype: int64


**Вывод:**

Создан датафрейм `market_file`, содержащий в себе данные о поведении покупателя на сайте, о коммуникациях с покупателем и его продуктовом поведении. Он имеет 13 столбцов и 1300 строк.

Столбцы имеют следующий характер:

- `id` — номер покупателя в корпоративной базе данных. Имеет тип `int64`, а значения распределены от 215348 до 216647.
- `Покупательская активность` — рассчитанный класс покупательской активности (целевой признак): `снизилась` или `прежний уровень`. Имеет тип `object` и 2 уникальных значения.
- `Тип сервиса` — уровень сервиса, например `премиум` и `стандарт`. Имеет тип `object` и 3 уникальных значения.
- `Разрешить сообщать` — информация о том, можно ли присылать покупателю дополнительные предложения о товаре. Согласие на это даёт покупатель. Имеет тип `object` и 2 уникальных значения.
- `Маркет_актив_6_мес` — среднемесячное значение маркетинговых коммуникаций компании, которое приходилось на покупателя за последние 6 месяцев. Это значение показывает, какое число рассылок, звонков, показов рекламы и прочего приходилось на клиента. Имеет тип `float64`, а значения распределены от 0.9 до 6.6, при этом среднее — примерно 4.25.
- `Маркет_актив_тек_мес` — количество маркетинговых коммуникаций в текущем месяце. Имеет тип `int64`, а значения распределены от 3 до 5, при этом среднее — примерно 4.
- `Длительность` — значение, которое показывает, сколько дней прошло с момента регистрации покупателя на сайте. Имеет тип `int64`, а значения распределены от 110 до 1079 дней, при этом среднее — примерно 602 дня.
- `Акционные_покупки` — среднемесячная доля покупок по акции от общего числа покупок за последние 6 месяцев. Имеет тип `float64`, а значения распределены от 0 до 0.99, при этом среднее — примерно 0.32.
- `Популярная_категория` — самая популярная категория товаров у покупателя за последние 6 месяцев. Имеет тип `object` и 6 уникальных значений.
- `Средний_просмотр_категорий_за_визит` — показывает, сколько в среднем категорий покупатель просмотрел за визит в течение последнего месяца. Имеет тип `int64`, а значения распределены от 1 до 6 категорий, при этом среднее — примерно 3 категории.
- `Неоплаченные_продукты_штук_квартал` — общее число неоплаченных товаров в корзине за последние 3 месяца. Имеет тип `int64`, а значения распределены от 0 до 10 товаров, при этом среднее — примерно 3 товара.
- `Ошибка_сервиса` — число сбоев, которые коснулись покупателя во время посещения сайта. Имеет тип `int64`, а значения распределены от 0 до 9 сбоев, при этом среднее — примерно 4 сбоя.
- `Страниц_за_визит` — среднее количество страниц, которые просмотрел покупатель за один визит на сайт за последние 3 месяца. Имеет тип `int64`, а значения распределены от 1 до 20 страниц, при этом среднее — примерно 8 страниц.

### Датасет `market_money.csv`

In [20]:
# создание датафрейма
market_money = create_dataframe('datasets/market_money.csv',
                                '/datasets/market_money.csv')

# обзор данных
data_review(market_money)

##### Первые 5 строк датасета

,id,Период,Выручка
0,215348,препредыдущий_месяц,0.0
1,215348,текущий_месяц,3293.1
2,215348,предыдущий_месяц,0.0
3,215349,препредыдущий_месяц,4472.0
4,215349,текущий_месяц,4971.6


##### Информация о столбцах и типах данных

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3900 entries, 0 to 3899
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id       3900 non-null   int64  
 1   Период   3900 non-null   object 
 2   Выручка  3900 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 91.5+ KB
None


##### Статистическое описание числовых столбцов

,id,Выручка
count,3900.000000,3900.000000
mean,215997.500000,5025.696051
std,375.325686,1777.704104
min,215348.000000,0.000000
25%,215672.750000,4590.150000
50%,215997.500000,4957.500000
75%,216322.250000,5363.000000
max,216647.000000,106862.200000


##### Кол-во уникальных значений столбцов типа object

Период    3
dtype: int64


**Вывод:**

Создан датафрейм `market_money`, содержащий в себе данные о выручке, которую получает магазин с покупателя (то есть сколько покупатель всего потратил за период взаимодействия с сайтом). Он имеет 3 столбца и 3900 строк.

Столбцы:

- `id` — номер покупателя в корпоративной базе данных. Имеет тип `int64`, а значения распределены от 215348 до 216647.
- `Период` — название периода, во время которого зафиксирована выручка. Например, `текущий_месяц` или `предыдущий_месяц`. Имеет тип `object` и 3 уникальных значения.
- `Выручка` — сумма выручки за период. Имеет тип `float64`, а значения распределены от 0 до 106862.2, при этом среднее — примерно 5025.7.

### Датасет `market_time.csv`

In [21]:
# создание датафрейма
market_time = create_dataframe('datasets/market_time.csv',
                               '/datasets/market_time.csv')

# обзор данных
data_review(market_time)

##### Первые 5 строк датасета

,id,Период,минут
0,215348,текущий_месяц,14
1,215348,предыдцщий_месяц,13
2,215349,текущий_месяц,10
3,215349,предыдцщий_месяц,12
4,215350,текущий_месяц,13


##### Информация о столбцах и типах данных

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2600 entries, 0 to 2599
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      2600 non-null   int64 
 1   Период  2600 non-null   object
 2   минут   2600 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 61.1+ KB
None


##### Статистическое описание числовых столбцов

,id,минут
count,2600.000000,2600.000000
mean,215997.500000,13.336154
std,375.349754,4.080198
min,215348.000000,4.000000
25%,215672.750000,10.000000
50%,215997.500000,13.000000
75%,216322.250000,16.000000
max,216647.000000,23.000000


##### Кол-во уникальных значений столбцов типа object

Период    2
dtype: int64


**Вывод:**

Создан датафрейм `market_time`, содержащий в себе данные о времени (в минутах), которое покупатель провёл на сайте в течение периода. Он имеет 3 столбца и 2600 строк.

Столбцы:

- `id` — номер покупателя в корпоративной базе данных. Имеет тип `int64`, а значения распределены от 215348 до 216647.
- `Период` — название периода, во время которого зафиксировано общее время. Имеет тип `object` и 2 уникальных значения.
- `минут` — значение времени, проведённого на сайте, в минутах. Имеет тип `int64`, а значения распределены от 4 до 23 мин, при этом среднее — примерно 13 мин.

### Датасет `money.csv`

In [22]:
# создание датафрейма
money = create_dataframe('datasets/money.csv',
                         '/datasets/money.csv',
                         sep=';', decimal=',')

# обзор данных
data_review(money)

##### Первые 5 строк датасета

,id,Прибыль
0,215348,0.98
1,215349,4.16
2,215350,3.13
3,215351,4.87
4,215352,4.21


##### Информация о столбцах и типах данных

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1300 entries, 0 to 1299
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id       1300 non-null   int64  
 1   Прибыль  1300 non-null   float64
dtypes: float64(1), int64(1)
memory usage: 20.4 KB
None


##### Статистическое описание числовых столбцов

,id,Прибыль
count,1300.000000,1300.000000
mean,215997.500000,3.996631
std,375.421985,1.013722
min,215348.000000,0.860000
25%,215672.750000,3.300000
50%,215997.500000,4.045000
75%,216322.250000,4.670000
max,216647.000000,7.430000


##### Кол-во уникальных значений столбцов типа object

Series([], dtype: float64)


**Вывод:**

Создан датафрейм `money`, содержащий в себе данные о среднемесячной прибыли покупателя за последние 3 месяца (какую прибыль получает магазин от продаж каждому покупателю). Он имеет 2 столбца и 1300 строк.

Столбцы:

- `id` — номер покупателя в корпоративной базе данных. Имеет тип `int64`, а значения распределены от 215348 до 216647.
- `Прибыль` — значение прибыли. Имеет тип `float64`, а значения распределены от 0.86 до 7.43, при этом среднее — примерно 4.

### Вывод загрузки данных

#### Датасет `market_file.csv`

Создан датафрейм `market_file`, содержащий в себе данные о поведении покупателя на сайте, о коммуникациях с покупателем и его продуктовом поведении. Он имеет 13 столбцов и 1300 строк.

Столбцы имеют следующий характер:

- `id` — номер покупателя в корпоративной базе данных. Имеет тип `int64`, а значения распределены от 215348 до 216647.
- `Покупательская активность` — рассчитанный класс покупательской активности (целевой признак): `снизилась` или `прежний уровень`. Имеет тип `object` и 2 уникальных значения.
- `Тип сервиса` — уровень сервиса, например `премиум` и `стандарт`. Имеет тип `object` и 3 уникальных значения.
- `Разрешить сообщать` — информация о том, можно ли присылать покупателю дополнительные предложения о товаре. Согласие на это даёт покупатель. Имеет тип `object` и 2 уникальных значения.
- `Маркет_актив_6_мес` — среднемесячное значение маркетинговых коммуникаций компании, которое приходилось на покупателя за последние 6 месяцев. Это значение показывает, какое число рассылок, звонков, показов рекламы и прочего приходилось на клиента. Имеет тип `float64`, а значения распределены от 0.9 до 6.6, при этом среднее — примерно 4.25.
- `Маркет_актив_тек_мес` — количество маркетинговых коммуникаций в текущем месяце. Имеет тип `int64`, а значения распределены от 3 до 5, при этом среднее — примерно 4.
- `Длительность` — значение, которое показывает, сколько дней прошло с момента регистрации покупателя на сайте. Имеет тип `int64`, а значения распределены от 110 до 1079 дней, при этом среднее — примерно 602 дня.
- `Акционные_покупки` — среднемесячная доля покупок по акции от общего числа покупок за последние 6 месяцев. Имеет тип `float64`, а значения распределены от 0 до 0.99, при этом среднее — примерно 0.32.
- `Популярная_категория` — самая популярная категория товаров у покупателя за последние 6 месяцев. Имеет тип `object` и 6 уникальных значений.
- `Средний_просмотр_категорий_за_визит` — показывает, сколько в среднем категорий покупатель просмотрел за визит в течение последнего месяца. Имеет тип `int64`, а значения распределены от 1 до 6 категорий, при этом среднее — примерно 3 категории.
- `Неоплаченные_продукты_штук_квартал` — общее число неоплаченных товаров в корзине за последние 3 месяца. Имеет тип `int64`, а значения распределены от 0 до 10 товаров, при этом среднее — примерно 3 товара.
- `Ошибка_сервиса` — число сбоев, которые коснулись покупателя во время посещения сайта. Имеет тип `int64`, а значения распределены от 0 до 9 сбоев, при этом среднее — примерно 4 сбоя.
- `Страниц_за_визит` — среднее количество страниц, которые просмотрел покупатель за один визит на сайт за последние 3 месяца. Имеет тип `int64`, а значения распределены от 1 до 20 страниц, при этом среднее — примерно 8 страниц.

#### Датасет `market_money.csv`

Создан датафрейм `market_money`, содержащий в себе данные о выручке, которую получает магазин с покупателя (то есть сколько покупатель всего потратил за период взаимодействия с сайтом). Он имеет 3 столбца и 3900 строк.

Столбцы:

- `id` — номер покупателя в корпоративной базе данных. Имеет тип `int64`, а значения распределены от 215348 до 216647.
- `Период` — название периода, во время которого зафиксирована выручка. Например, `текущий_месяц` или `предыдущий_месяц`. Имеет тип `object` и 3 уникальных значения.
- `Выручка` — сумма выручки за период. Имеет тип `float64`, а значения распределены от 0 до 106862.2, при этом среднее — примерно 5025.7.

#### Датасет `market_time.csv`

Создан датафрейм `market_time`, содержащий в себе данные о времени (в минутах), которое покупатель провёл на сайте в течение периода. Он имеет 3 столбца и 2600 строк.

Столбцы:

- `id` — номер покупателя в корпоративной базе данных. Имеет тип `int64`, а значения распределены от 215348 до 216647.
- `Период` — название периода, во время которого зафиксировано общее время. Имеет тип `object` и 2 уникальных значения.
- `минут` — значение времени, проведённого на сайте, в минутах. Имеет тип `int64`, а значения распределены от 4 до 23 мин, при этом среднее — примерно 13 мин.

#### Датасет `money.csv`

Создан датафрейм `money`, содержащий в себе данные о среднемесячной прибыли покупателя за последние 3 месяца (какую прибыль получает магазин от продаж каждому покупателю). Он имеет 2 столбца и 1300 строк.

Столбцы:

- `id` — номер покупателя в корпоративной базе данных. Имеет тип `int64`, а значения распределены от 215348 до 216647.
- `Прибыль` — значение прибыли. Имеет тип `float64`, а значения распределены от 0.86 до 7.43, при этом среднее — примерно 4.

## Предобработка данных

### Переименование столбцов

#### Датафрейм `market_file`

In [ ]:
market_file = rename_columns(market_file)

id - id
Покупательская активность - buying_activity
Тип сервиса - service_type
Разрешить сообщать - allow_reporting
Маркет_актив_6_мес - market_active_6_months
Маркет_актив_тек_мес - market_active_tech_mes
Длительность - duration


**Вывод:**

Названия столбцов датафрейма `market_file` переименованы согласно PEP8. Новые названия столбцов:

- `id` — номер покупателя в корпоративной базе данных;
- `buying_activity` — рассчитанный класс покупательской активности;
- `service_type` — уровень сервиса;
- `allow_reporting` — информация о том, можно ли присылать покупателю дополнительные предложения о товаре;
- `market_active_6_months` — среднемесячное значение маркетинговых коммуникаций компании, которое приходилось на покупателя за последние 6 месяцев;
- `market_active_tech_mes` — количество маркетинговых коммуникаций в текущем месяце;
- `duration` — значение, которое показывает, сколько дней прошло с момента регистрации покупателя на сайте;
- `promotional_purchases` — среднемесячная доля покупок по акции от общего числа покупок за последние 6 месяцев;
- `popular_category` — самая популярная категория товаров у покупателя за последние 6 месяцев;
- `average_category_views_per_visit` — показывает, сколько в среднем категорий покупатель просмотрел за визит в течение последнего месяца;
- `unpaid_products_pieces_quarter` — общее число неоплаченных товаров в корзине за последние 3 месяца;
- `service_error` — число сбоев, которые коснулись покупателя во время посещения сайта;
- `pages_per_visit` — среднее количество страниц, которые просмотрел покупатель за один визит на сайт за последние 3 месяца.

#### Датафрейм `market_money`

In [ ]:
market_money = rename_columns(market_money)

**Вывод:**

Названия столбцов датафрейма `market_money` переименованы согласно PEP8. Новые названия столбцов:

- `id` — номер покупателя в корпоративной базе данных;
- `period` — название периода, во время которого зафиксирована выручка;
- `revenue` — сумма выручки за период.

#### Датафрейм `market_time`

In [ ]:
market_time = rename_columns(market_time)

**Вывод:**

Названия столбцов датафрейма `market_time` переименованы согласно PEP8. Новые названия столбцов:

- `id` — номер покупателя в корпоративной базе данных;
- `period` — название периода, во время которого зафиксировано общее время;
- `minutes` — значение времени, проведённого на сайте, в минутах.

#### Датафрейм `money`

In [ ]:
money = rename_columns(money)

**Вывод:**

Названия столбцов датафрейма `money` переименованы согласно PEP8. Новые названия столбцов:

- `id` — номер покупателя в корпоративной базе данных;
- `profit` — значение прибыли.

### Проверка типов данных

#### Датафрейм `market_file`

Типы данных:

In [ ]:
market_file.dtypes

Изменение типов данных:

In [ ]:
# словарь, который показывает как именно
# изменить типы данных в датафрейме
column_types = {
    'market_active_6_months': 'float32',
    'promotional_purchases': 'float32'
}

# изменение типов данных
market_file = change_data_types(market_file,
                                column_types)

**Вывод:**

В датафрейме `market_file` все столбцы имеют корректные типы данных. Однако значения столбцов `market_active_6_months`, `promotional_purchases` были переведены из `float64` в `float32` для экономии вычислительной памяти.

#### Датафрейм `market_money`

Типы данных:

In [ ]:
market_money.dtypes

Изменение типов данных:

In [ ]:
# словарь, который показывает как именно
# изменить типы данных в датафрейме
column_types = {
    'revenue': 'float32'
}

# изменение типов данных
market_money = change_data_types(market_money,
                                 column_types)

**Вывод:**

В датафрейме `market_money` все столбцы имеют корректные типы данных. Однако значения столбца `revenue` были переведены из `float64` в `float32` для экономии вычислительной памяти.

#### Датафрейм `market_time`

Типы данных:

In [ ]:
market_time.dtypes

**Вывод:**

В датафрейме `market_time` все столбцы имеют корректные типы данных.

#### Датафрейм `money`

Типы данных:

In [ ]:
money.dtypes

Изменение типов данных:

In [ ]:
# словарь, который показывает как именно
# изменить типы данных в датафрейме
column_types = {
    'profit': 'float32'
}

# изменение типов данных
money = change_data_types(money,
                          column_types)

**Вывод:**

В датафрейме `money` все столбцы имеют корректные типы данных. Однако значения столбца `profit` были переведены из `float64` в `float32` для экономии вычислительной памяти.

### Изучение пропущенных значений

#### Датафрейм `market_file`

Отображение пропущенных значений:

In [ ]:
msno.bar(market_file, color='#597dbf');

**Вывод:**

В датафрейме `market_file` пропущенных значений не наблюдается.

#### Датафрейм `market_money`

Отображение пропущенных значений:

In [ ]:
msno.bar(market_money, color='#597dbf');

**Вывод:**

В датафрейме `market_money` пропущенных значений не наблюдается.

#### Датафрейм `market_time`

Отображение пропущенных значений:

In [ ]:
msno.bar(market_time, color='#597dbf');

**Вывод:**

В датафрейме `market_time` пропущенных значений не наблюдается.

#### Датафрейм `money`

Отображение пропущенных значений:

In [ ]:
msno.bar(money, color='#597dbf');

**Вывод:**

В датафрейме `money` пропущенных значений не наблюдается.

### Изучение дубликатов

#### Датафрейм `market_file`

Уникальные значения столбцов типа `object`:

In [ ]:
object_unique(market_file)

Замена строковых значений:

In [ ]:
# исправлние опечаток
market_file['service_type'] = (market_file['service_type']
                               .str.replace('стандартт', 'стандарт'))
market_file['popular_category'] = (market_file['popular_category']
                                   .str.replace('Косметика и аксесуары', 'Косметика и аксессуары'))

# перевод в нижний регистр
market_file['buying_activity'] = market_file['buying_activity'].str.lower()
market_file['popular_category'] = market_file['popular_category'].str.lower()

# проверка
object_unique(market_file)

Количество явных дубликатов:

In [ ]:
# интересующие столбцы
cols_mf = list(market_file.columns)
cols_mf.remove('id')

# вывод дубликатов
obvious_duplicates(market_file, [cols_mf])

Удаление явных дубликатов:

In [ ]:
market_file = market_file.drop_duplicates(subset=cols_mf).reset_index(drop=True)

# проверка
obvious_duplicates(market_file, [cols_mf])

**Вывод:**

В столбце `service_type` датафрейма `market_file` были выявлены неявные дубликаты, поэтому их строковые значения были заменены следующим образом: `стандартт` на `стандарт`. Также в столбце `popular_category` исправлена опечатка: `Косметика и аксесуары` на `Косметика и аксессуары`. Еще значения в столбцах `buying_activity` и `popular_category` были приведены к нижнему регистру. При этом было выявлено и удалено 11 явных дубликата по всем столбцам кроме `id`.

#### Датафрейм `market_money`

Уникальные значения столбцов типа `object`:

In [ ]:
object_unique(market_money)

Замена строковых значений:

In [ ]:
# исправлние опечатки
market_money['period'] = (market_money['period']
                          .str.replace('препредыдущий_месяц',
                                       'предпредыдущий_месяц'))

# проверка
object_unique(market_money)

Количество явных дубликатов:

In [ ]:
obvious_duplicates(market_money, [['id', 'period']])

**Вывод:**

В столбце `period` датафрейма `market_money` была найдена опечатка, поэтому строковые значения были заменены следующим образом: `препредыдущий_месяц` на `предпредыдущий_месяц`. При этом неявных и явных дубликатов не обнаружено.

#### Датафрейм `market_time`

Уникальные значения столбцов типа `object`:

In [ ]:
object_unique(market_time)

Замена строковых значений:

In [ ]:
# исправлние опечатки
market_time['period'] = (market_time['period']
                         .str.replace('предыдцщий_месяц',
                                      'предыдущий_месяц'))

# проверка
object_unique(market_time)

Количество явных дубликатов:

In [ ]:
obvious_duplicates(market_time, [['id', 'period']])

**Вывод:**

В столбце `period` датафрейма `market_time` была найдена опечатка, поэтому строковые значения были заменены следующим образом: `предыдцщий_месяц` на `предыдущий_месяц`. При этом неявных и явных дубликатов не обнаружено.

#### Датафрейм `money`

Количество явных дубликатов:

In [ ]:
obvious_duplicates(money)

**Вывод:**

В датафрейме `money` неявных и явных дубликатов не обнаружено.

### Вывод предобработки данных

#### Переименование столбцов

Названия столбцов датафрейма `market_file` переименованы согласно PEP8. Новые названия столбцов:

- `id` — номер покупателя в корпоративной базе данных;
- `buying_activity` — рассчитанный класс покупательской активности;
- `service_type` — уровень сервиса;
- `allow_reporting` — информация о том, можно ли присылать покупателю дополнительные предложения о товаре;
- `market_active_6_months` — среднемесячное значение маркетинговых коммуникаций компании, которое приходилось на покупателя за последние 6 месяцев;
- `market_active_tech_mes` — количество маркетинговых коммуникаций в текущем месяце;
- `duration` — значение, которое показывает, сколько дней прошло с момента регистрации покупателя на сайте;
- `promotional_purchases` — среднемесячная доля покупок по акции от общего числа покупок за последние 6 месяцев;
- `popular_category` — самая популярная категория товаров у покупателя за последние 6 месяцев;
- `average_category_views_per_visit` — показывает, сколько в среднем категорий покупатель просмотрел за визит в течение последнего месяца;
- `unpaid_products_pieces_quarter` — общее число неоплаченных товаров в корзине за последние 3 месяца;
- `service_error` — число сбоев, которые коснулись покупателя во время посещения сайта;
- `pages_per_visit` — среднее количество страниц, которые просмотрел покупатель за один визит на сайт за последние 3 месяца.

Названия столбцов датафрейма `market_money` переименованы согласно PEP8. Новые названия столбцов:

- `id` — номер покупателя в корпоративной базе данных;
- `period` — название периода, во время которого зафиксирована выручка;
- `revenue` — сумма выручки за период.

Названия столбцов датафрейма `market_time` переименованы согласно PEP8. Новые названия столбцов:

- `id` — номер покупателя в корпоративной базе данных;
- `period` — название периода, во время которого зафиксировано общее время;
- `minutes` — значение времени, проведённого на сайте, в минутах.

Названия столбцов датафрейма `money` переименованы согласно PEP8. Новые названия столбцов:

- `id` — номер покупателя в корпоративной базе данных;
- `profit` — значение прибыли.

#### Проверка типов данных

В датафрейме `market_file` все столбцы имеют корректные типы данных. Однако значения столбцов `market_active_6_months`, `promotional_purchases` были переведены из `float64` в `float32` для экономии вычислительной памяти.

В датафрейме `market_money` все столбцы имеют корректные типы данных. Однако значения столбца `revenue` были переведены из `float64` в `float32` для экономии вычислительной памяти.

В датафрейме `market_time` все столбцы имеют корректные типы данных.

В датафрейме `money` все столбцы имеют корректные типы данных. Однако значения столбца `profit` были переведены из `float64` в `float32` для экономии вычислительной памяти.

#### Изучение пропущенных значений

В датафреймах `market_file`, `market_money`, `market_time` и `money` пропущенных значений не наблюдается.

#### Изучение дубликатов

В столбце `service_type` датафрейма `market_file` были выявлены неявные дубликаты, поэтому их строковые значения были заменены следующим образом: `стандартт` на `стандарт`. Также в столбце `popular_category` исправлена опечатка: `Косметика и аксесуары` на `Косметика и аксессуары`. Еще значения в столбцах `buying_activity` и `popular_category` были приведены к нижнему регистру. При этом было выявлено и удалено 11 явных дубликата по всем столбцам кроме `id`.

В столбце `period` датафрейма `market_money` была найдена опечатка, поэтому строковые значения были заменены следующим образом: `препредыдущий_месяц` на `предпредыдущий_месяц`. При этом неявных и явных дубликатов не обнаружено.

В столбце `period` датафрейма `market_time` была найдена опечатка, поэтому строковые значения были заменены следующим образом: `предыдцщий_месяц` на `предыдущий_месяц`. При этом неявных и явных дубликатов не обнаружено.

В датафрейме `money` неявных и явных дубликатов не обнаружено.

## Исследовательский анализ данных

### Коммуникация с пользователем

#### Тип сервиса

In [ ]:
cat_plots(market_file, column='service_type', hue='buying_activity',
          legend_title='Типы', title='Тип сервиса')

**Вывод:**

Большинство пользователей предпочитают стандартный тип сервиса как при снижении покупательской активности (65.3%, 318 пользователей), так и при её сохранении на прежнем уровне (74.3%, 596 пользователей). Причем снижение покупательской активности приводит к увеличению доли клиентов (на 9%), использующих премиум-сервис.

#### Разрешение на сообщения

In [ ]:
cat_plots(market_file, column='allow_reporting', hue='buying_activity',
          legend_title='Разрешения', title='Разрешение на сообщения')

**Вывод:**

Большинство пользователей предпочитают включать разрешения на сообщения как при снижении покупательской активности (74.7%, 364 пользователя), так и при её сохранении на прежнем уровне (73.7%, 591 пользователь).

#### Маркетинг за 6 месяцев

Вывод статистического описания:

In [ ]:
more_describe(market_file, column='market_active_6_months',
              hue='buying_activity')

Составление графиков:

In [ ]:
num_plots(market_file, x='market_active_6_months', hue='buying_activity', bins=30,
          xlabel='среднемесячное значение маркетинговых коммуникаций',
          ylabel='кол-во пользователей',
          legend_title='Покуп. активность',
          title='Маркетинг за 6 месяцев')

**Вывод:**

Распределение маркетинга за 6 месяцев более симметрично у пользователей со сниженной покупательской активностью. Обе группы в большинстве своем имеют среднемесячное значение маркетинговых коммуникаций примерно равное 4. При этом у обоих распределений максимальное значение составляет 6.6, а минимальное — 0.9. В предоставленных данных, пользователи с прежним уровнем покупательской активности имеют большее среднемесячное значение маркетинговых коммуникаций (медиана — 4.4) по сравнению с пользователями, у которых активность снизилась (медиана — 3.9). При этом межквартильные размахи у обоих групп пользователей практически совпадают (у сниженного уровня — 1.3, а у прежнего — 1.27), что указывает на примерно одинаковое распределение данных. Также в сниженном уровне покупательской активности наблюдается 2.26% выбросов (в значениях меньше 1 или больше 6), а в прежнем — 0.25% (в значениях меньше 2).

#### Маркетинг за текущий месяц

Вывод статистического описания:

In [ ]:
more_describe(market_file, column='market_active_tech_mes',
              hue='buying_activity')

Составление графиков:

In [ ]:
num_plots(market_file, x='market_active_tech_mes', hue='buying_activity',
          xlabel='кол-во маркетинговых коммуникаций',
          ylabel='кол-во пользователей',
          legend_title='Покуп. активность',
          title='Маркетинг за текущий месяц')

**Вывод:**

Распределение маркетинга за текущий месяц более симметрично у пользователей со сниженной покупательской активностью. Обе группы в большинстве своем имеют количество маркетинговых коммуникаций равное 4. При этом у обоих распределений максимальное значение составляет 5, а минимальное — 3. В предоставленных данных, пользователи обоих групп имеют одинаковое кол-во маркетинговых коммуникаций (медиана — 4). Межквартильный размах у пользователей с прежним уровнем покупательской активности слишком мал (0), и по этой причине отображается огромная доля выбросов (47.5%). У сниженного же уровня покупательской активности никаких выбросов не наблюдается, а межквартильный размах находится в диапазоне от 4 до 5 маркетинговых коммуникаций.

#### Длительность истории

Вывод статистического описания:

In [ ]:
more_describe(market_file, column='duration',
              hue='buying_activity')

Составление графиков:

In [ ]:
num_plots(market_file, x='duration', hue='buying_activity', bins=30,
          xlabel='кол-во дней с момента регистрации покупателя на сайте',
          ylabel='кол-во пользователей',
          legend_title='Покуп. активность',
          title='Длительность истории')

**Вывод:**

Распределение длительности истории является многомодальным у обоих групп пользователей и имеет пики примерно в 500 и 800 дней. У пользователей со сниженной покупательской активностью минимальное значение составляет 110 дней, а максимальное — 1079 дней. С прежним уровнем — 121 и 1061 дней соответсвтенно. В предоставленных данных, пользователи со сниженным уровнем покупательской активности зарегистрированы на сайте дольше (медиана — 637 дней) по сравнению с пользователями, у которых активность сохранилась на прежнем уровне (медиана — 590 дней). При этом межквартильный размах для пользователей с прежним уровнем покупательской активности шире (на 54 дня), что указывает на более широкое распределение данных. Выбросов в обоих группах не обнаружено.

### Продуктовое поведение

#### Наиболее частая категория продукта в заказе

In [ ]:
cat_plots(market_file, column='popular_category', hue='buying_activity',
          legend_title='Категории', title='Наиболее частая категория продукта в заказе')

**Вывод:**

Товары для детей являются наиболее частой категорией продуктов в заказах для обеих групп пользователей, но их доля выше (на 6.5%) среди пользователей с пониженной покупательской активностью. Косметика, аксессуары и домашний текстиль также популярны среди пользователей с пониженной активностью, но их доли ниже среди пользователей с прежним уровнем покупательской активности (косметика и аксессуары — на 6.7%, домашний текстиль — на 1.9%). Мелкая бытовая техника и электроника значительно более популярна среди пользователей с прежним уровнем покупательской активности по сравнению с пользователями с пониженной (на 13.2%).

#### Среднее количество просматриваемых категорий продуктов за визит

Вывод статистического описания:

In [ ]:
more_describe(market_file, column='average_category_views_per_visit',
              hue='buying_activity')

Составление графиков:

In [ ]:
num_plots(market_file, x='average_category_views_per_visit', hue='buying_activity',
          xlabel='среднее кол-во просматриваемых категорий',
          ylabel='кол-во пользователей',
          legend_title='Покуп. активность',
          title='Среднее кол-во просматриваемых категорий продуктов за визит')

**Вывод:**

Распределение среднего количества просматриваемых категорий продуктов за визит более симметрично у пользователей с прежним уровнем покупательской активности, при этом в большинстве своем они имеют среднее кол-во просматриваемых категорий равное 3 и 4. У сниженного же уровня покупательской активности наиболее частое значение — 2, что значительно меньше. Также у обоих распределений максимальное значение составляет 6, а минимальное — 1. В предоставленных данных, пользователи с прежним уровнем покупательской активности просматривают в 2 раза больше категорий продуктов за визит (медиана — 4) по сравнению с пользователями, у которых покупательская активность снизилась (медиана — 2). При этом межквартильный размах для пользователей с прежним уровнем покупательской активности шире (на 1), что указывает на более широкое распределение данных. Также в сниженном уровне покупательской активности наблюдается 8.42% выбросов (в значениях больше или равно 5), а в прежнем — они отсутствуют.

#### Количество неоплаченных продуктов в корзине за последние 3 месяца

Вывод статистического описания:

In [ ]:
more_describe(market_file, column='unpaid_products_pieces_quarter',
              hue='buying_activity')

Составление графиков:

In [ ]:
num_plots(market_file, x='unpaid_products_pieces_quarter', hue='buying_activity',
          xlabel='кол-во неоплаченных продуктов',
          ylabel='кол-во пользователей',
          legend_title='Покуп. активность',
          title='Кол-во неоплаченных продуктов в корзине за последние 3 месяца')

**Вывод:**

Распределение количества неоплаченных продуктов в корзине за последние 3 месяца более симметрично у пользователей со сниженным уровнем покупательской активности, при этом в большинстве своем они имеют кол-во неоплаченных продуктов в размере от 2 до 5. У прежнего же уровня покупательской активности наиболее частое значение — 1, что значительно меньше. Также у обоих распределений минимальные значения совпадают (в значении 0), а максимальные расходятся (у сниженного уровня — 10, а у прежнего — 8). В предоставленных данных, пользователи с прежним уровнем покупательской активности имеют в 2 раза меньше неоплаченных продуктов в корзине (медиана — 2) по сравнению с пользователями, у которых покупательская активность снизилась (медиана — 4). При этом межквартильный размах для пользователей со сниженным уровнем покупательской активности шире (на 1), что указывает на более широкое распределение данных. Также в сниженном уровне покупательской активности наблюдается 0.82% выбросов (в значениях больше или равно 10), а в прежнем — 0.37% (в значениях больше или равно 7).

### Финансовое поведение

Создание сводной таблицы по выручке за разные периоды:

In [ ]:
market_money_pivot = market_money.pivot(index='id',
                                        columns='period',
                                        values='revenue')
market_money_pivot = market_money_pivot.merge(market_file[['id', 'buying_activity']],
                                              on='id', how='inner')

# проверка
market_money_pivot.head(10)

#### Разница в выручке предыдущего и предпредыдущего месяца

Подсчет разницы в выручке предыдущего и предпредыдущего месяца:

In [ ]:
market_money_pivot['revenue_diff_prem_preprem'] = abs(market_money_pivot['предпредыдущий_месяц'] -\
                                                      market_money_pivot['предыдущий_месяц'])

Вывод статистического описания:

In [ ]:
more_describe(market_money_pivot, column='revenue_diff_prem_preprem',
              hue='buying_activity')

Составление графиков:

In [ ]:
num_plots(market_money_pivot, x='revenue_diff_prem_preprem', hue='buying_activity', bins=30,
          xlabel='разница в выручке',
          ylabel='кол-во пользователей',
          legend_title='Покуп. активность',
          title='Разница в выручке предыдущего и предпредыдущего месяца')

**Вывод:**

Распределение разницы в выручке предыдущего и предпредыдущего месяца более симметрично у пользователей со сниженным уровнем покупательской активности, при этом в большинстве своем они имеют разницу в выручке в размере примерно 428. У прежнего же уровня покупательской активности наиболее частое значение — примерно 214, что значительно меньше. Также у обоих распределений минимальные значения совпадают 0, а максимальные расходятся (у сниженного уровня — 2067, а у прежнего — 1999). В предоставленных данных, пользователи с прежним уровнем покупательской активности имеют меньшую разницу в выручке (медиана — 392.5) по сравнению с пользователями, у которых покупательская активность снизилась (медиана — 657). При этом межквартильный размах для пользователей со сниженным уровнем покупательской активности шире (на 145.63), что указывает на более широкое распределение данных. Также в сниженном уровне покупательской активности наблюдается 0.2% выбросов (в значениях больше 2000), а в прежнем — 0.37% (в значениях больше или равно 1500).

#### Разница в выручке текущего и предыдущего месяца

Подсчет разницы в выручке текущего и предыдущего месяца:

In [ ]:
market_money_pivot['revenue_diff_curm_prem'] = abs(market_money_pivot['предыдущий_месяц'] -\
                                                   market_money_pivot['текущий_месяц'])

Вывод статистического описания:

In [ ]:
more_describe(market_money_pivot, column='revenue_diff_curm_prem',
              hue='buying_activity')

Составление графиков:

In [ ]:
num_plots(market_money_pivot, x='revenue_diff_curm_prem', hue='buying_activity',
          bins=30,
          xlabel='разница в выручке',
          ylabel='кол-во пользователей',
          legend_title='Покуп. активность',
          title='Разница в выручке текущего и предыдущего месяца')

Удаление пользователя с аномальным значением и повторный подсчет выбросов:

In [ ]:
# удаление пользователя с аномальным значением
abnormal_id = market_money[market_money['revenue'] > 100000]['id']
condition = ~market_money['id'].isin(abnormal_id)
market_money = market_money[condition]

# повторное создание сводной таблицы по выручке за разные периоды
market_money_pivot = market_money.pivot(index='id', columns='period',
                                        values='revenue')
market_money_pivot = market_money_pivot.merge(market_file[['id', 'buying_activity']],
                                              on='id', how='inner')

# повторный подсчет разниц в выручках
market_money_pivot['revenue_diff_prem_preprem'] = abs(market_money_pivot['предпредыдущий_месяц'] -\
                                                      market_money_pivot['предыдущий_месяц'])
market_money_pivot['revenue_diff_curm_prem'] = abs(market_money_pivot['предыдущий_месяц'] -\
                                                   market_money_pivot['текущий_месяц'])

# повторный вывод статистического описания
more_describe(market_money_pivot, column='revenue_diff_curm_prem',
              hue='buying_activity')

Повторное составление графиков:

In [ ]:
num_plots(market_money_pivot, x='revenue_diff_curm_prem', hue='buying_activity',
          bins=80, xlim=(0, 2000),
          xlabel='разница в выручке',
          ylabel='кол-во пользователей',
          legend_title='Покуп. активность',
          title='Разница в выручке текущего и предыдущего месяца')

**Вывод:**

Распределение разницы в выручке текущего и предыдущего месяца более симметрично у пользователей со сниженным уровнем покупательской активности, при этом в большинстве своем они имеют разницу в выручке в размере примерно 250. У прежнего же уровня покупательской активности наиболее частое значение — примерно 167, что значительно меньше. Также у обоих распределений минимальные (у сниженного уровня — 1.3, а у прежнего — 0) и максимальные (у сниженного уровня — 5986.3, а у прежнего — 1446.9) значения не совпадают. В предоставленных данных, обе группы пользователей имеют примерно одинаковые разницы в выручке текущего и предыдущего месяца (медиана у сниженного уровня — 388.7, а у прежнего — 373.1). При этом межквартильный размах для пользователей с прежним уровнем покупательской активности шире (на 57.2), что указывает на более широкое распределение данных. Также в сниженном уровне покупательской активности было удалено одно аномальное значение (100785.2), причем после удаления наблюдается 1.65% выбросов (в значениях больше 1300). В прежнем же уровне — 0.25% выбросов (в значениях больше 1400).

#### Выручка в текущем месяце

Вывод статистического описания:

In [ ]:
more_describe(market_money_pivot, column='текущий_месяц',
              hue='buying_activity')

Составление графиков:

In [ ]:
num_plots(market_money_pivot, x='текущий_месяц', hue='buying_activity', bins=30,
          xlabel='выручка',
          ylabel='кол-во пользователей',
          legend_title='Покуп. активность',
          title='Выручка за текущий месяц')

**Вывод:**

Распределение выручки за текущий месяц более симметрично у пользователей с прежним уровнем покупательской активности, при этом в большинстве своем они имеют выручку в размере примерно 5000. У сниженного же уровня покупательской активности наиболее частое значение — примерно 5333. Также у обоих распределений минимальные (у сниженного уровня — 2758.7, а у прежнего — 2952.2) и максимальные (у сниженного уровня — 7799.4, а у прежнего — 7547.8) значения не совпадают. В предоставленных данных, пользователи с прежним уровнем покупательской активности имеют меньшую выручку (медиана — 5122.55) по сравнению с пользователями, у которых покупательская активность снизилась (медиана — 5296.7). При этом межквартильный размах для пользователей со сниженным уровнем покупательской активности шире (на 436.25), что указывает на более широкое распределение данных. Также в сниженном уровне покупательской активности выбросов не обнаружилось, а в прежнем — 4.11% выбросов (в значениях меньше 3500 или больше 6800).

### Поведение на сайте

Создание сводной таблицы по времени на сайте за разные периоды:

In [ ]:
market_time_pivot = market_time.pivot(index='id',
                                      columns='period',
                                      values='minutes')
market_time_pivot = market_time_pivot.merge(market_file[['id', 'buying_activity']],
                                            on='id', how='inner')

# проверка
market_time_pivot.head(10)

#### Время, проведенное на сайте за предыдущий месяц

Вывод статистического описания:

In [ ]:
more_describe(market_time_pivot, column='предыдущий_месяц',
              hue='buying_activity')

Составление графиков:

In [ ]:
num_plots(market_time_pivot, x='предыдущий_месяц', hue='buying_activity', bins=10,
          xlabel='минуты',
          ylabel='кол-во пользователей',
          legend_title='Покуп. активность',
          title='Время, проведенное на сайте за предыдущий месяц')

**Вывод:**

Распределение времени, проведенного на сайте за предыдущий месяц, более равномерно у пользователей со сниженным уровнем покупательской активности, при этом в большинстве своем они имеют время примерно равное 10 мин. У прежнего же уровня покупательской активности наиболее частое значение — примерно 15 мин. Также у обоих распределений минимальные значения не совпадают (у сниженного уровня — 5 мин, а у прежнего — 7 мин), а максимальные наоборот — совпадают (23 мин). В предоставленных данных, пользователи с прежним уровнем покупательской активности в прошлом месяце провели больше времени за сайтом (медиана — 15 мин) по сравнению с пользователями, у которых покупательская активность снизилась (медиана — 10 мин). При этом межквартильные размахи пользователей обоих групп совпадают (5 мин), что указывает на примерно одинаковое распределение данных. Также в прежнем уровне покупательской активности выбросов не обнаружилось, а в сниженном — 1.44% выбросов (в значениях больше 20 мин).

#### Время, проведенное на сайте за текущий месяц

Вывод статистического описания:

In [ ]:
more_describe(market_time_pivot, column='текущий_месяц',
              hue='buying_activity')

Составление графиков:

In [ ]:
num_plots(market_time_pivot, x='текущий_месяц', hue='buying_activity', bins=10,
          xlabel='минуты',
          ylabel='кол-во пользователей',
          legend_title='Покуп. активность',
          title='Время, проведенное на сайте за текущий месяц')

**Вывод:**

Распределение времени, проведенного на сайте за текущий месяц, более равномерно у пользователей со сниженным уровнем покупательской активности, при этом в большинстве своем они имеют время примерно равное от 8 до 11 мин. У прежнего же уровня покупательской активности наиболее частое значение — примерно 15 мин. Также у обоих распределений минимальные значения совпадают (4 мин), а максимальные наоборот — не совпадают (у сниженного уровня — 22 мин, а у прежнего — 23 мин). В предоставленных данных, пользователи с прежним уровнем покупательской активности в текущем месяце провели больше времени за сайтом (медиана — 15 мин) по сравнению с пользователями, у которых покупательская активность снизилась (медиана — 10 мин). При этом межквартильный размах для пользователей с прежним уровнем покупательской активности шире (на 2 мин), что указывает на более широкое распределение данных. Также в прежнем уровне покупательской активности выбросов не обнаружилось, а в сниженном — 2.46% выбросов (в значениях больше или равно 20 мин).

#### Количество просматриваемых страниц за визит

Вывод статистического описания:

In [ ]:
more_describe(market_file, column='pages_per_visit',
              hue='buying_activity')

Составление графиков:

In [ ]:
num_plots(market_file, x='pages_per_visit', hue='buying_activity', bins=10,
          xlabel='кол-во страниц',
          ylabel='кол-во пользователей',
          legend_title='Покуп. активность',
          title='Кол-во просматриваемых страниц за визит')

**Вывод:**

Распределение количества просматриваемых страниц за визит более симметрично у пользователей с прежним уровнем покупательской активности, при этом в большинстве своем они просматривают 10 страниц. У сниженного же уровня покупательской активности наиболее частое значение — 3-4 страницы, что значительно ниже. Также у обоих распределений минимальные (у сниженного уровня — 1 стр., а у прежнего — 3 стр.) и максимальные (у сниженного уровня — 18 стр., а у прежнего — 20 стр.) значения не совпадают. В предоставленных данных, пользователи с прежним уровнем покупательской активности просматривают в 2 раза больше страниц (медиана — 10 стр.) по сравнению с пользователями, у которых покупательская активность снизилась (медиана — 5 стр.). При этом межквартильный размах для пользователей с прежним уровнем покупательской активности шире (на 1 стр.), что указывает на более широкое распределение данных. Также в сниженном уровне покупательской активности наблюдается 3.9% выбросов (в значениях больше или равно 14 стр.), а в прежнем — 0.25% (в значениях больше или равно 20 стр.).

#### Количество сбоев сайта

Вывод статистического описания:

In [ ]:
more_describe(market_file, column='service_error',
              hue='buying_activity')

Составление графиков:

In [ ]:
num_plots(market_file, x='service_error', hue='buying_activity', bins=10,
          xlabel='кол-во сбоев',
          ylabel='кол-во пользователей',
          legend_title='Покуп. активность',
          title='Кол-во сбоев сайта')

**Вывод:**

Распределение количества сбоев сайта более симметрично у пользователей с прежним уровнем покупательской активности, при этом в большинстве своем они имеют 4 сбоя. У сниженного же уровня покупательской активности наиболее частое значение — 3 сбоя. Также у обоих распределений минимальные значения не совпадают (у сниженного уровня — 1 сбой, а у прежнего — 0 сбоев), а максимальные наоборот — совпадают (9 сбоев). В предоставленных данных, пользователи обоих групп имеют одинаковое количество сбоев сайта (медиана — 4 сбоя). И их межквартильные размахи тоже совпадают (3 сбоя), что указывает на примерно одинаковое распределение данных. Также выбросов в обоих группах не замечено.

### Отбор пользователей с покупательской активностью не менее трёх месяцев

Определение `id` пользователей, которые за последние 3 месяца хотя бы в одном месяце не принесли выручку:

In [ ]:
# пересоздание сводной таблицы по выручке
market_money_pivot = market_money.pivot(index='id',
                                        columns='period',
                                        values='revenue')

# определение неактивных пользователей по выручке
id_not_active_clients = market_money_pivot[(market_money_pivot['предпредыдущий_месяц'] == 0) |\
                                           (market_money_pivot['предыдущий_месяц'] == 0) |\
                                           (market_money_pivot['текущий_месяц'] == 0)].index

# преобразование в список
id_not_active_clients = list(id_not_active_clients)

# вывод id пользователей
id_not_active_clients

Удаление пользователей, которые за последние 3 месяца хотя бы в одном месяце не принесли выручку:

In [ ]:
# в датафрейме market_file
condition1 = ~market_file['id'].isin(id_not_active_clients)
market_file = market_file[condition1]

# в датафрейме market_money
condition2 = ~market_money['id'].isin(id_not_active_clients)
market_money = market_money[condition2]

# в датафрейме market_time
condition3 = ~market_time['id'].isin(id_not_active_clients)
market_time = market_time[condition3]

# в датафрейме money
condition4 = ~money['id'].isin(id_not_active_clients)
money = money[condition4]

# проверка
if (condition1.any() and condition2.any() and \
    condition3.any() and condition4.any()):
    print('Пользователи удалены')
else:
    print('Пользователи не удалены')

**Вывод:**

Во всех датафреймах были удалены пользователи (`id`: 215348, 215357, 215359), которые за последние 3 месяца хотя бы в одном месяце не принесли выручку.

### Покупательская активность

In [ ]:
piechart(market_file['buying_activity'],
         title='Соотношение покупательской активности',
         legend_title='Покупательская активность',
         figsize=(10, 8))

**Вывод:**

В значениях покупательской активности наблюдается небольшой дисбаланс. Доля пользователей с прежним уровнем покупательской активности составляет 62.4% (802 пользователя), а со сниженным уровнем — 37.6% (484 пользователя). В свою очередь это может отразиться на обучении модели машинного обучения и её метриках качества.

### Вывод исследовательского анализа данных

#### Коммуникация с пользователем

Большинство пользователей предпочитают стандартный тип сервиса как при снижении покупательской активности (65.3%, 318 пользователей), так и при её сохранении на прежнем уровне (74.3%, 596 пользователей). Причем снижение покупательской активности приводит к увеличению доли клиентов (на 9%), использующих премиум-сервис.

Большинство пользователей предпочитают включать разрешения на сообщения как при снижении покупательской активности (74.7%, 364 пользователя), так и при её сохранении на прежнем уровне (73.7%, 591 пользователь).

Распределение маркетинга за 6 месяцев более симметрично у пользователей со сниженной покупательской активностью. Обе группы в большинстве своем имеют среднемесячное значение маркетинговых коммуникаций примерно равное 4. При этом у обоих распределений максимальное значение составляет 6.6, а минимальное — 0.9. В предоставленных данных, пользователи с прежним уровнем покупательской активности имеют большее среднемесячное значение маркетинговых коммуникаций (медиана — 4.4) по сравнению с пользователями, у которых активность снизилась (медиана — 3.9). При этом межквартильные размахи у обоих групп пользователей практически совпадают (у сниженного уровня — 1.3, а у прежнего — 1.27), что указывает на примерно одинаковое распределение данных. Также в сниженном уровне покупательской активности наблюдается 2.26% выбросов (в значениях меньше 1 или больше 6), а в прежнем — 0.25% (в значениях меньше 2).

Распределение маркетинга за текущий месяц более симметрично у пользователей со сниженной покупательской активностью. Обе группы в большинстве своем имеют количество маркетинговых коммуникаций равное 4. При этом у обоих распределений максимальное значение составляет 5, а минимальное — 3. В предоставленных данных, пользователи обоих групп имеют одинаковое кол-во маркетинговых коммуникаций (медиана — 4). Межквартильный размах у пользователей с прежним уровнем покупательской активности слишком мал (0), и по этой причине отображается огромная доля выбросов (47.5%). У сниженного же уровня покупательской активности никаких выбросов не наблюдается, а межквартильный размах находится в диапазоне от 4 до 5 маркетинговых коммуникаций.

Распределение длительности истории является многомодальным у обоих групп пользователей и имеет пики примерно в 500 и 800 дней. У пользователей со сниженной покупательской активностью минимальное значение составляет 110 дней, а максимальное — 1079 дней. С прежним уровнем — 121 и 1061 дней соответсвтенно. В предоставленных данных, пользователи со сниженным уровнем покупательской активности зарегистрированы на сайте дольше (медиана — 637 дней) по сравнению с пользователями, у которых активность сохранилась на прежнем уровне (медиана — 590 дней). При этом межквартильный размах для пользователей с прежним уровнем покупательской активности шире (на 54 дня), что указывает на более широкое распределение данных. Выбросов в обоих группах не обнаружено.

#### Продуктовое поведение

Товары для детей являются наиболее частой категорией продуктов в заказах для обеих групп пользователей, но их доля выше (на 6.5%) среди пользователей с пониженной покупательской активностью. Косметика, аксессуары и домашний текстиль также популярны среди пользователей с пониженной активностью, но их доли ниже среди пользователей с прежним уровнем покупательской активности (косметика и аксессуары — на 6.7%, домашний текстиль — на 1.9%). Мелкая бытовая техника и электроника значительно более популярна среди пользователей с прежним уровнем покупательской активности по сравнению с пользователями с пониженной (на 13.2%).

Распределение среднего количества просматриваемых категорий продуктов за визит более симметрично у пользователей с прежним уровнем покупательской активности, при этом в большинстве своем они имеют среднее кол-во просматриваемых категорий равное 3 и 4. У сниженного же уровня покупательской активности наиболее частое значение — 2, что значительно меньше. Также у обоих распределений максимальное значение составляет 6, а минимальное — 1. В предоставленных данных, пользователи с прежним уровнем покупательской активности просматривают в 2 раза больше категорий продуктов за визит (медиана — 4) по сравнению с пользователями, у которых покупательская активность снизилась (медиана — 2). При этом межквартильный размах для пользователей с прежним уровнем покупательской активности шире (на 1), что указывает на более широкое распределение данных. Также в сниженном уровне покупательской активности наблюдается 8.42% выбросов (в значениях больше или равно 5), а в прежнем — они отсутствуют.

Распределение количества неоплаченных продуктов в корзине за последние 3 месяца более симметрично у пользователей со сниженным уровнем покупательской активности, при этом в большинстве своем они имеют кол-во неоплаченных продуктов в размере от 2 до 5. У прежнего же уровня покупательской активности наиболее частое значение — 1, что значительно меньше. Также у обоих распределений минимальные значения совпадают (в значении 0), а максимальные расходятся (у сниженного уровня — 10, а у прежнего — 8). В предоставленных данных, пользователи с прежним уровнем покупательской активности имеют в 2 раза меньше неоплаченных продуктов в корзине (медиана — 2) по сравнению с пользователями, у которых покупательская активность снизилась (медиана — 4). При этом межквартильный размах для пользователей со сниженным уровнем покупательской активности шире (на 1), что указывает на более широкое распределение данных. Также в сниженном уровне покупательской активности наблюдается 0.82% выбросов (в значениях больше или равно 10), а в прежнем — 0.37% (в значениях больше или равно 7).

#### Финансовое поведение

Распределение разницы в выручке предыдущего и предпредыдущего месяца более симметрично у пользователей со сниженным уровнем покупательской активности, при этом в большинстве своем они имеют разницу в выручке в размере примерно 428. У прежнего же уровня покупательской активности наиболее частое значение — примерно 214, что значительно меньше. Также у обоих распределений минимальные значения совпадают 0, а максимальные расходятся (у сниженного уровня — 2067, а у прежнего — 1999). В предоставленных данных, пользователи с прежним уровнем покупательской активности имеют меньшую разницу в выручке (медиана — 392.5) по сравнению с пользователями, у которых покупательская активность снизилась (медиана — 657). При этом межквартильный размах для пользователей со сниженным уровнем покупательской активности шире (на 145.63), что указывает на более широкое распределение данных. Также в сниженном уровне покупательской активности наблюдается 0.2% выбросов (в значениях больше 2000), а в прежнем — 0.37% (в значениях больше или равно 1500).

Распределение разницы в выручке текущего и предыдущего месяца более симметрично у пользователей со сниженным уровнем покупательской активности, при этом в большинстве своем они имеют разницу в выручке в размере примерно 250. У прежнего же уровня покупательской активности наиболее частое значение — примерно 167, что значительно меньше. Также у обоих распределений минимальные (у сниженного уровня — 1.3, а у прежнего — 0) и максимальные (у сниженного уровня — 5986.3, а у прежнего — 1446.9) значения не совпадают. В предоставленных данных, обе группы пользователей имеют примерно одинаковые разницы в выручке текущего и предыдущего месяца (медиана у сниженного уровня — 388.7, а у прежнего — 373.1). При этом межквартильный размах для пользователей с прежним уровнем покупательской активности шире (на 57.2), что указывает на более широкое распределение данных. Также в сниженном уровне покупательской активности было удалено одно аномальное значение (100785.2), причем после удаления наблюдается 1.65% выбросов (в значениях больше 1300). В прежнем же уровне — 0.25% выбросов (в значениях больше 1400).

Распределение выручки за текущий месяц более симметрично у пользователей с прежним уровнем покупательской активности, при этом в большинстве своем они имеют выручку в размере примерно 5000. У сниженного же уровня покупательской активности наиболее частое значение — примерно 5333. Также у обоих распределений минимальные (у сниженного уровня — 2758.7, а у прежнего — 2952.2) и максимальные (у сниженного уровня — 7799.4, а у прежнего — 7547.8) значения не совпадают. В предоставленных данных, пользователи с прежним уровнем покупательской активности имеют меньшую выручку (медиана — 5122.55) по сравнению с пользователями, у которых покупательская активность снизилась (медиана — 5296.7). При этом межквартильный размах для пользователей со сниженным уровнем покупательской активности шире (на 436.25), что указывает на более широкое распределение данных. Также в сниженном уровне покупательской активности выбросов не обнаружилось, а в прежнем — 4.11% выбросов (в значениях меньше 3500 или больше 6800).

#### Поведение на сайте

Распределение времени, проведенного на сайте за предыдущий месяц, более равномерно у пользователей со сниженным уровнем покупательской активности, при этом в большинстве своем они имеют время примерно равное 10 мин. У прежнего же уровня покупательской активности наиболее частое значение — примерно 15 мин. Также у обоих распределений минимальные значения не совпадают (у сниженного уровня — 5 мин, а у прежнего — 7 мин), а максимальные наоборот — совпадают (23 мин). В предоставленных данных, пользователи с прежним уровнем покупательской активности в прошлом месяце провели больше времени за сайтом (медиана — 15 мин) по сравнению с пользователями, у которых покупательская активность снизилась (медиана — 10 мин). При этом межквартильные размахи пользователей обоих групп совпадают (5 мин), что указывает на примерно одинаковое распределение данных. Также в прежнем уровне покупательской активности выбросов не обнаружилось, а в сниженном — 1.44% выбросов (в значениях больше 20 мин).

Распределение времени, проведенного на сайте за текущий месяц, более равномерно у пользователей со сниженным уровнем покупательской активности, при этом в большинстве своем они имеют время примерно равное от 8 до 11 мин. У прежнего же уровня покупательской активности наиболее частое значение — примерно 15 мин. Также у обоих распределений минимальные значения совпадают (4 мин), а максимальные наоборот — не совпадают (у сниженного уровня — 22 мин, а у прежнего — 23 мин). В предоставленных данных, пользователи с прежним уровнем покупательской активности в текущем месяце провели больше времени за сайтом (медиана — 15 мин) по сравнению с пользователями, у которых покупательская активность снизилась (медиана — 10 мин). При этом межквартильный размах для пользователей с прежним уровнем покупательской активности шире (на 2 мин), что указывает на более широкое распределение данных. Также в прежнем уровне покупательской активности выбросов не обнаружилось, а в сниженном — 2.46% выбросов (в значениях больше или равно 20 мин).

Распределение количества просматриваемых страниц за визит более симметрично у пользователей с прежним уровнем покупательской активности, при этом в большинстве своем они просматривают 10 страниц. У сниженного же уровня покупательской активности наиболее частое значение — 3-4 страницы, что значительно ниже. Также у обоих распределений минимальные (у сниженного уровня — 1 стр., а у прежнего — 3 стр.) и максимальные (у сниженного уровня — 18 стр., а у прежнего — 20 стр.) значения не совпадают. В предоставленных данных, пользователи с прежним уровнем покупательской активности просматривают в 2 раза больше страниц (медиана — 10 стр.) по сравнению с пользователями, у которых покупательская активность снизилась (медиана — 5 стр.). При этом межквартильный размах для пользователей с прежним уровнем покупательской активности шире (на 1 стр.), что указывает на более широкое распределение данных. Также в сниженном уровне покупательской активности наблюдается 3.9% выбросов (в значениях больше или равно 14 стр.), а в прежнем — 0.25% (в значениях больше или равно 20 стр.).

Распределение количества сбоев сайта более симметрично у пользователей с прежним уровнем покупательской активности, при этом в большинстве своем они имеют 4 сбоя. У сниженного же уровня покупательской активности наиболее частое значение — 3 сбоя. Также у обоих распределений минимальные значения не совпадают (у сниженного уровня — 1 сбой, а у прежнего — 0 сбоев), а максимальные наоборот — совпадают (9 сбоев). В предоставленных данных, пользователи обоих групп имеют одинаковое количество сбоев сайта (медиана — 4 сбоя). И их межквартильные размахи тоже совпадают (3 сбоя), что указывает на примерно одинаковое распределение данных. Также выбросов в обоих группах не замечено.

#### Отбор пользователей с покупательской активностью не менее трех месяцев

Во всех датафреймах были удалены пользователи (`id`: 215348, 215357, 215359), которые за последние 3 месяца хотя бы в одном месяце не принесли выручку.

#### Покупательская активность

В значениях покупательской активности наблюдается небольшой дисбаланс. Доля пользователей с прежним уровнем покупательской активности составляет 62.4% (802 пользователя), а со сниженным уровнем — 37.6% (484 пользователя). В свою очередь это может отразиться на обучении модели машинного обучения и её метриках качества.

## Объединение данных

Пересоздание сводной таблицы `market_money_pivot` (так как был отбор пользователей):

In [ ]:
# сводная таблица
market_money_pivot = market_money.pivot(index='id',
                                        columns='period',
                                        values='revenue').reset_index()
market_money_pivot.columns.name = None

# переименование столбцов
market_money_pivot = rename_columns(market_money_pivot, ending='_revenue',
                                    col_no_ending=['id'], inf=False)

# проверка
market_money_pivot.head(10)

Пересоздание сводной таблицы `market_time_pivot` (так как был отбор пользователей):

In [ ]:
# сводная таблица
market_time_pivot = market_time.pivot(index='id',
                                      columns='period',
                                      values='minutes').reset_index()
market_time_pivot.columns.name = None

# переименование столбцов
market_time_pivot = rename_columns(market_time_pivot, ending='_time',
                                   col_no_ending=['id'], inf=False)

# проверка
market_time_pivot.head(10)

Объединение датафреймов:

In [ ]:
market_fm = market_file.merge(market_money_pivot, on='id', how='inner')
market = market_fm.merge(market_time_pivot, on='id', how='inner')

# проверка
print(f'Размеры исходных датафреймов: {market_file.shape}, {market_money_pivot.shape} и {market_time_pivot.shape}')
print(f'Размер датафрейма, полученного в результате объединения: {market.shape}')

Вывод первых десяти строк:

In [ ]:
market.head(10)

**Вывод:**

Было произведено объединение датафреймов `market_file` (1286, 13), `market_money_pivot` (1296, 4) и `market_time_pivot` (1297, 3) в один датафрейм `market` (1285, 18).

## Корреляционный анализ

### Анализ с помощью коэффициента $\phi_k$

Составление корреляционной тепловой карты, используя коэффициент $\phi_k$:

In [ ]:
# интересующие столбцы
cols = market.columns.to_list()
cols.remove('id')

# столбцы с непрерывными значениями
interval_cols = ['market_active_6_months', 'duration',
                 'promotional_purchases', 'preprevious_month_revenue',
                 'previous_month_revenue', 'current_month_revenue',
                 'previous_month_time', 'current_month_time']

# построение корр. тепловой карты
corr_heatmap(market[cols], interval_cols=interval_cols,
             title='Матрица корреляций phik')

**Вывод:**

Согласно коэффициенту $\phi_k$, в датафрейме `market` наблюдаются следующие сильные корреляции:

- между целевым признаком `buying_activity` и: `pages_per_visit` (0.8), `previous_month_time` (0.7);
- между входными признаками: `current_month_revenue` и `previous_month_revenue` (0.8).

Очень высоких корреляций (> 0.9) между входными признаками не наблюдается, скорее всего мультиколлинеарности нет.

Составление корреляционных тепловых карт для каждой категории целевого признака `buying_activity`, используя коэффициент $\phi_k$:

In [ ]:
# удаление buying_activity
# из интересующих столбцов
cols.remove('buying_activity')

# построение корр. тепловой карты
# для сниженной покупательской активности
corr_heatmap(market[cols][market['buying_activity'] == 'снизилась'],
             interval_cols=interval_cols,
             title='Матрица корреляций phik для сниженной покупательской активности')

# построение корр. тепловой карты
# для прежней покупательской активности
corr_heatmap(market[cols][market['buying_activity'] == 'прежний уровень'],
             interval_cols=interval_cols,
             title='Матрица корреляций phik для прежней покупательской активности')

**Вывод:**

Никаких сильных новых корреляций для каждой категории целевого признака `buying_activity` не выявлено.

### Анализ с помощью диаграмм рассеяния

Построение сетки диаграмм рассеяния:

In [ ]:
# интересующие столбцы
cols = market.columns.to_list()

# построение диаграмм рассеивания
g = sns.PairGrid(market[cols], hue='buying_activity')
g.map_diag(sns.histplot)
g.map_offdiag(sns.scatterplot)
g.add_legend();

Отображение зависимости между выручками за текущий и прошлый месяцы:

In [ ]:
scatter_plot(df=market,
             x='current_month_revenue',
             y='previous_month_revenue',
             hue='buying_activity',
             xlabel='выручка за текущий месяц',
             ylabel='выручка за прошлый месяц',
             legend_title='Покуп. активность',
             title='Зависимость между выручками за текущий и прошлый месяцы')

Отображение зависимости между `id` и акционными покупками:

In [ ]:
scatter_plot(df=market,
             x='id',
             y='promotional_purchases',
             hue='buying_activity',
             xlabel='id',
             ylabel='акционные покупки',
             legend_title='Покуп. активность',
             title='Зависимость между id и акционными покупками')

**Вывод:**

Наблюдается явная линейная зависимость между выручками за текущий и прошлый месяцы. Также замечена довольно значительная часть пользователей, которые за последние полгода большую часть покупок совершали по акции, причем у средних `id` отсутствует такое поведение.

#### Зависимость между выручками в различных группах

Объединение выручек за позапрошлый и прошлый месяцы:

In [ ]:
# создание копии
market_copy = market.copy()

# объединение выручек
market_copy['revenue_preprem_prem'] = (market_copy['preprevious_month_revenue'] +
                                       market_copy['previous_month_revenue'])

# проверка
market_copy[['preprevious_month_revenue',
             'previous_month_revenue',
             'revenue_preprem_prem']].head(10)

Отображение зависимости между выручками по покупательской активности:

In [ ]:
scatter_plot(df=market_copy,
             x='current_month_revenue',
             y='revenue_preprem_prem',
             hue='buying_activity',
             xlabel='выручка за текущий месяц',
             ylabel='выручка за позапрошлый и прошлый месяцы',
             legend_title='Покуп. активность',
             title='Зависимость между выручками по покупательской активности')

Отображение зависимости между выручками по типу сервиса:

In [ ]:
scatter_plot(df=market_copy,
             x='current_month_revenue',
             y='revenue_preprem_prem',
             hue='service_type',
             xlabel='выручка за текущий месяц',
             ylabel='выручка за позапрошлый и прошлый месяцы',
             legend_title='Типы сервиса',
             title='Зависимость между выручками по типу сервиса')

Отображение зависимости между выручками по разрешениям на сообщения:

In [ ]:
scatter_plot(df=market_copy,
             x='current_month_revenue',
             y='revenue_preprem_prem',
             hue='allow_reporting',
             xlabel='выручка за текущий месяц',
             ylabel='выручка за позапрошлый и прошлый месяцы',
             legend_title='Разрешения',
             title='Зависимость между выручками по разрешениям на сообщения')

Отображение зависимости между выручками по наиболее частой категории продукта в заказе:

In [ ]:
scatter_plot(df=market_copy,
             x='current_month_revenue',
             y='revenue_preprem_prem',
             hue='popular_category',
             xlabel='выручка за текущий месяц',
             ylabel='выручка за позапрошлый и прошлый месяцы',
             legend_title='Категории',
             title='Зависимость между выручками по наиболее частой категории продукта в заказе')

**Вывод:**

Наблюдается сильная линейная зависимость между выручками у пользователей со сниженной покупательской активностью, с премиум-сервисом и с популярной категорией "товары для детей".

#### Зависимость между `id` и акционными покупками в различных группах

Отображение зависимости между `id` и акционными покупками по покупательской активности:

In [ ]:
scatter_plot(df=market,
             x='id',
             y='promotional_purchases',
             hue='buying_activity',
             xlabel='id',
             ylabel='акционные покупки',
             legend_title='Покуп. активность',
             title='Зависимость между id и акционными покупками по покупательской активности')

Отображение зависимости между `id` и акционными покупками по типу сервиса:

In [ ]:
scatter_plot(df=market,
             x='id',
             y='promotional_purchases',
             hue='service_type',
             xlabel='id',
             ylabel='акционные покупки',
             legend_title='Тип сервиса',
             title='Зависимость между id и акционными покупками по типу сервиса')

Отображение зависимости между `id` и акционными покупками по разрешениям на сообщения:

In [ ]:
scatter_plot(df=market,
             x='id',
             y='promotional_purchases',
             hue='allow_reporting',
             xlabel='id',
             ylabel='акционные покупки',
             legend_title='Разрешения',
             title='Зависимость между id и акционными покупками по разрешениям на сообщения')

Отображение зависимости между `id` и акционными покупками по наиболее частой категории продукта в заказе:

In [ ]:
scatter_plot(df=market,
             x='id',
             y='promotional_purchases',
             hue='popular_category',
             xlabel='id',
             ylabel='акционные покупки',
             legend_title='Разрешения',
             title='Зависимость между id и акционными покупками по наиболее частой категории продукта в заказе')

**Вывод:**

Большинство пользователей, которые за последние полгода чаще всего совершали покупки по акциям, отличаются сниженной покупательской активностью, используют стандартный сервис и разрешают получать сообщения.

### Вывод корреляционного анализа

#### Анализ с помощью коэффициента $\phi_k$

Согласно коэффициенту $\phi_k$, в датафрейме `market` наблюдаются следующие сильные корреляции:

- между целевым признаком `buying_activity` и: `pages_per_visit` (0.8), `previous_month_time` (0.7);
- между входными признаками: `current_month_revenue` и `previous_month_revenue` (0.8).

Очень высоких корреляций (> 0.9) между входными признаками не наблюдается, скорее всего мультиколлинеарности нет.

Также никаких сильных новых корреляций для каждой категории целевого признака `buying_activity` не выявлено.

#### Анализ с помощью диаграмм рассеяния

Наблюдается явная линейная зависимость между выручками за текущий и прошлый месяцы. Также замечена довольно значительная часть пользователей, которые за последние полгода большую часть покупок совершали по акции, причем у средних `id` отсутствует такое поведение.

Наблюдается сильная линейная зависимость между выручками у пользователей со сниженной покупательской активностью, с премиум-сервисом и с популярной категорией "товары для детей".

Большинство пользователей, которые за последние полгода чаще всего совершали покупки по акциям, отличаются сниженной покупательской активностью, используют стандартный сервис и разрешают получать сообщения.

## Построение модели машинного обучения

### Выбор признаков

Входные признаки:

In [ ]:
# категориальные входные признаки
ohe_columns = ['service_type', 'allow_reporting']
ord_columns = ['popular_category']

# численные входные признаки
num_columns = cols
num_columns.remove('id')
num_columns.remove('buying_activity')
for col in ohe_columns + ord_columns:
    num_columns.remove(col)

# вывод входных признаков
print(f'Категориальные входные признаки для OneHotEncoding:\n{ohe_columns}\n')
print(f'Категориальные входные признаки для OrdinalEncoding:\n{ord_columns}\n')
print(f'Численные входные признаки:\n{num_columns}\n')

Разделение данных на выборки:

In [ ]:
# входные признаки
X = market[['id'] + ohe_columns + ord_columns + num_columns]

# кодирование целевого признака
label_encoder = LabelEncoder()
label_encoder.fit(['прежний уровень', 'снизилась'])
y = label_encoder.transform(market.buying_activity)

# проверка кодирования целевого признака
print('Целевой признак закодирован следующим образом:')
print(f'0 - {list(label_encoder.classes_)[0]}')
print(f'1 - {list(label_encoder.classes_)[1]}')

# разделение на выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=RANDOM_STATE,
                                                    test_size=0.25, stratify=y)

**Вывод:**

В качестве целевого признака был выбран столбец `buying_activity`, а в качестве входных — остальные столбцы из датафрейма `market`, кроме `id`. Также было произведено кодирование (label encoder) целевого признака (прежний уровень покупательской активности обозначается как `0`, а сниженный — как `1`). Затем данные были разделены на тренировочную (75%) и тестовую выборки (25%) с применением стратификации.

### Пайплайны

Пайплайн по кодированию:

In [ ]:
# для подготовки признаков из списка ohe_columns
ohe_pipe = Pipeline(
    [
        (
            'simpleImputer_ohe', 
            SimpleImputer(missing_values=np.nan, strategy='most_frequent')
        ),
        (
            'ohe', 
            OneHotEncoder(drop='first', handle_unknown='ignore')
        )
    ]
)

# для подготовки признаков из списка ord_columns
ord_pipe = Pipeline(
    [
        (
            'simpleImputer_before_ord', 
            SimpleImputer(missing_values=np.nan, strategy='most_frequent')
        ),
        (
            'ord',
            OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=np.nan)
        ),
        (
            'simpleImputer_after_ord', 
            SimpleImputer(missing_values=np.nan, strategy='most_frequent')
        )
    ]
)

Пайплайн по подготовке данных:

In [ ]:
preprocessor = ColumnTransformer(
    [
        ('ohe', ohe_pipe, ohe_columns),
        ('ord', ord_pipe, ord_columns),
        ('num', StandardScaler(), num_columns)
    ],
    remainder='passthrough'
)

Итоговый пайплайн:

In [ ]:
pipe_final= Pipeline([
    ('preprocessor', preprocessor),
    ('models', LogisticRegression(random_state=RANDOM_STATE))
])

Задание кодирования, масштабирования, моделей и их гиперпараметров:

In [ ]:
param = [
    # словарь для модели KNeighborsClassifier() 
    {
        'models': [KNeighborsClassifier()],
        'models__n_neighbors': range(2, 11),
        'models__metric': ['manhattan', 'euclidean'],
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), RobustScaler(), 'passthrough']   
    },
    # словарь для модели DecisionTreeClassifier()
    {
        'models': [DecisionTreeClassifier(random_state=RANDOM_STATE)],
        'models__max_depth': range(2, 11),
        'models__min_samples_split': range(2, 11),
        'models__min_samples_leaf': range(1, 11),
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), RobustScaler(), 'passthrough']  
    },
    # словарь для модели SVC()
    {
        'models': [SVC(random_state=RANDOM_STATE)],
        'models__C': range(1, 11),
        'models__kernel': ['linear', 'rbf', 'sigmoid', 'poly'],
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), RobustScaler(), 'passthrough']
    },
    # словарь для модели LogisticRegression()
    {
        'models': [LogisticRegression(solver='liblinear', penalty='l1',
                                      random_state=RANDOM_STATE)],
        'models__C': range(1, 11),
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), RobustScaler(), 'passthrough']  
    }
]

**Вывод:**

Для подготовки данных были выбраны следующие методы:

- кодирование: `OneHotEncoder()` и `OrdinalEncoder()`;
- масштабирование: `StandardScaler()`, `MinMaxScaler()` и `RobustScaler()`.

Для задачи классификации было выбрано 4 модели машинного обучения:

- `KNeighborsClassifier()` — метод k-ближайших соседей с гиперпараметрами: кол-во соседей от 2 до 10 и манхеттанское, евклидово расстояния;
- `DecisionTreeClassifier()` — дерево решений с гиперпараметрами: максимальная глубина дерева от 2 до 10, минимальное кол-во объектов в узле от 2 до 10 и минимальное кол-во объектов в листе от 1 до 10;
- `SVC()` — метод опорных векторов с гиперпараметрами: сила регуляризации от 1 до 10 и линейное, радиальная базисная функция, сигмоидное, полиномиальное ядра;
- `LogisticRegression()` — логистическая регрессия с l1-регуляризацией с гиперпараметром сила регуляризации от 1 до 10.

### Кросс-валидация

Для оценки качества модели была выбрана метрика `ROC-AUC`, так как она не зависит от порога классификации, учитывает как `True Positive Rate`, так и `False Positive Rate`, что делает её более устойчивой к дисбалансу классов по сравнению с метриками, такими как `accuracy`.

Случайная кросс-валидация:

In [ ]:
start_time = time.time()

randomized_search = RandomizedSearchCV(
    pipe_final, 
    param, 
    scoring='roc_auc',
    cv=5,
    random_state=RANDOM_STATE,
    n_jobs=-1
)

randomized_search.fit(X_train[ohe_columns + ord_columns + num_columns],
                      y_train)

end_time = time.time()

execution_time = round(end_time - start_time, 2)

# сохранение лучшей модели
# и значения метрики на валидационных данных
model = randomized_search.best_estimator_
roc_auc_val = np.round(randomized_search.best_score_, 2)

# преобразование тренировочной и тестовой выборок
X_train_prepr = model.named_steps['preprocessor'].transform(X_train[ohe_columns + ord_columns + num_columns])
X_test_prepr = model.named_steps['preprocessor'].transform(X_test[ohe_columns + ord_columns + num_columns])

# значение метрики на тестовых данных
y_pred = model.named_steps['models'].predict(X_test_prepr)
y_proba = model.named_steps['models'].predict_proba(X_test_prepr)[:, 1]
roc_auc_test = np.round(roc_auc_score(y_test, y_proba), 2)

# вывод значений метрик
print(f'Лучшая модель: {model}')
print(f'ROC-AUC на валидационных данных: {roc_auc_val}')
print(f'ROC-AUC на тестовых данных: {roc_auc_test}')
print(f'Время выполнения: {execution_time}')

**Вывод:**

При случайной кросс-валидации лучшей моделью оказалось логистическая регрессия с l1-регуляризацией и гиперпараметром `C` равным 9. Причем категориальные данные подготавливались с помощью кодировщиков `OneHotEncoding()` (`service_type` и `allow_reporting`) и `OrdinalEncoder()` (`popular_category`), а численные остались неизменными. Значение `ROC-AUC` лучшей модели на валидационных данных составляет 0.9, а на тестовых — 0.91.

### Анализ модели

Матрица ошибок лучшей модели:

In [ ]:
conf_matrix(y_test, y_pred, le=label_encoder,
            title='Матрица ошибок лучшей модели')

Вывод других метрик лучшей модели:

In [ ]:
# метрики
best_accuracy = accuracy_score(y_test, y_pred)
best_precision = precision_score(y_test, y_pred)
best_recall = recall_score(y_test, y_pred)

# вывод
print(f'accuracy = {round(best_accuracy, 2):.2f}')
print(f'precision = {round(best_precision, 2):.2f}')
print(f'recall = {round(best_recall, 2):.2f}')

**Вывод:**

Матрица ошибок лучшей модели имеет следующий вид:

- 190 пользователей с прежним уровнем покупательской активности были верно классифицированы (TN),
- 22 пользователя со сниженным уровнем покупательской активности были неверно классифицированы (FN),
- 99 пользователей со сниженным уровнем покупательской активности были верно классифицированы (TP),
- 11 пользователей с прежним уровнем покупательской активности был неверно классифицирован (FP).

Доля верных прогнозов на тестовой выборке составляет 90% (accuracy), причем стоит учесть, что в предоставленных данных пользователей с прежним уровнем покупательской активности на 24.8% больше, чем со сниженным уровнем. Точность же, с которой модель присваивает пользователям категорию `снизилась` составляет 90% (precision). Причем 82% (recall) пользователей категории `прежний уровень` модель смогла правильно идентифицировать среди всех пользователей категории `прежний уровень`.

### Графики важности SHAP

График важности SHAP в виде плотбара:

In [ ]:
# получение названий признаков после предобработки
feature_names = list(model.named_steps['preprocessor'].get_feature_names_out())

# создание объяснителя SHAP
explainer1 = shap.LinearExplainer(model.named_steps['models'], X_train_prepr)

# получение SHAP значений
shap_values1 = explainer1(X_test_prepr)

# создание объекта Explanation с указанием имен признаков
shap_values_with_names = shap.Explanation(shap_values1,
                                          feature_names=feature_names)

# график важности SHAP в виде плотбара
shap.plots.bar(shap_values_with_names, max_display=16)

График важности SHAP в виде beeswarm:

In [ ]:
shap.plots.beeswarm(shap_values_with_names, max_display=17)

# создание объяснителя SHAP
#shap_sample_train = shap.sample(X_train_prepr, 100)
#shap_sample_test = shap.sample(X_test_prepr, 100)
#explainer2 = shap.KernelExplainer(lambda x: model.named_steps['models'].predict_proba(x),
                                  #shap_sample_train)

# получение SHAP значений
#shap_values2 = explainer2(shap_sample_test)
#shap_values_transformed = shap.Explanation(values=shap_values2.values[:,:,1],
                                           #base_values=explainer2.expected_value[1],
                                           #data=shap_sample_test,
                                           #feature_names=feature_names)

# построение графика beeswarm
#shap.plots.beeswarm(shap_values_transformed, max_display=17)

**Вывод:**

С точки зрения модели и согласно графику важности SHAP в виде плотбара, входные признаки можно разделить на:

- **значимые**: среднее кол-во просмотренных страниц за визит (0.63), проведенное время на сайте за разные периоды (предыдущий месяц — 0.62, текущий месяц — 0.39), среднее количество просматриваемых категорий продуктов за визит (0.48), кол-во неоплаченных продуктов в корзине (0.47), среднемесячная доля покупок по акции от общего числа покупок (0.41);
- **малозначимые**: среднемесячное значение маркетинговых коммуникаций (0.34), выручки за разные периоды (предпредыдущий месяц — 0.18, предыдущий месяц — 0.26, текущий месяц — 0.25), популярная категория (0.17);
- **незначимые**: длительность истории с клиентом (0.09), число сбоев сайта (0.06), кол-во маркетинговых коммуникаций в текущем месяце (0.03), разрешения на сообщения (0.02), тип сервиса (0.01).

Также согласно графику важности SHAP в виде beeswarm, модель говорит о том, что высокие значения среднего кол-ва просмотренных страниц за визит, проведенного времени на сайте за разные периоды, среднего количества просматриваемых категорий продуктов за визит и низкие значения кол-ва неоплаченных продуктов в корзине, среднемесячной доли покупок по акции от общего числа покупок влияют на определение прежнего уровня покупательской активности.

Следовательно, интернет-магазину «В один клик» следует обращать внимание на низкие значения среднего кол-ва просмотренных страниц за визит, проведенного времени на сайте за разные периоды, среднего количества просматриваемых категорий продуктов за визит и высокие значения кол-ва неоплаченных продуктов в корзине, среднемесячной доли покупок по акции от общего числа покупок, так как все они, с точки зрения модели, значительно влияют на определение сниженного уровня покупательской активности.

### Вывод построения модели машинного обучения

#### Выбор признаков

В качестве целевого признака был выбран столбец `buying_activity`, а в качестве входных — остальные столбцы из датафрейма `market`, кроме `id`. Также было произведено кодирование (label encoder) целевого признака (прежний уровень покупательской активности обозначается как `0`, а сниженный — как `1`). Затем данные были разделены на тренировочную (75%) и тестовую выборки (25%) с применением стратификации.

#### Пайплайны

Для подготовки данных были выбраны следующие методы:

- кодирование: `OneHotEncoder()` и `OrdinalEncoder()`;
- масштабирование: `StandardScaler()`, `MinMaxScaler()` и `RobustScaler()`.

Для задачи классификации было выбрано 4 модели машинного обучения:

- `KNeighborsClassifier()` — метод k-ближайших соседей с гиперпараметрами: кол-во соседей от 2 до 10 и манхеттанское, евклидово расстояния;
- `DecisionTreeClassifier()` — дерево решений с гиперпараметрами: максимальная глубина дерева от 2 до 10, минимальное кол-во объектов в узле от 2 до 10 и минимальное кол-во объектов в листе от 1 до 10;
- `SVC()` — метод опорных векторов с гиперпараметрами: сила регуляризации от 1 до 10 и линейное, радиальная базисная функция, сигмоидное, полиномиальное ядра;
- `LogisticRegression()` — логистическая регрессия с l1-регуляризацией с гиперпараметром сила регуляризации от 1 до 10.

#### Кросс-валидация

Для оценки качества модели была выбрана метрика `ROC-AUC`, так как она не зависит от порога классификации, учитывает как `True Positive Rate`, так и `False Positive Rate`, что делает её более устойчивой к дисбалансу классов по сравнению с метриками, такими как `accuracy`.

При случайной кросс-валидации лучшей моделью оказалось логистическая регрессия с l1-регуляризацией и гиперпараметром `C` равным 9. Причем категориальные данные подготавливались с помощью кодировщиков `OneHotEncoding()` (`service_type` и `allow_reporting`) и `OrdinalEncoder()` (`popular_category`), а численные остались неизменными. Значение `ROC-AUC` лучшей модели на валидационных данных составляет 0.9, а на тестовых — 0.91.

#### Анализ модели

Матрица ошибок лучшей модели имеет следующий вид:

- 190 пользователей с прежним уровнем покупательской активности были верно классифицированы (TN),
- 22 пользователя со сниженным уровнем покупательской активности были неверно классифицированы (FN),
- 99 пользователей со сниженным уровнем покупательской активности были верно классифицированы (TP),
- 11 пользователей с прежним уровнем покупательской активности был неверно классифицирован (FP).

Доля верных прогнозов на тестовой выборке составляет 90% (accuracy), причем стоит учесть, что в предоставленных данных пользователей с прежним уровнем покупательской активности на 24.8% больше, чем со сниженным уровнем. Точность же, с которой модель присваивает пользователям категорию `снизилась` составляет 90% (precision). Причем 82% (recall) пользователей категории `прежний уровень` модель смогла правильно идентифицировать среди всех пользователей категории `прежний уровень`.

#### Графики важности SHAP

С точки зрения модели и согласно графику важности SHAP в виде плотбара, входные признаки можно разделить на:

- **значимые**: среднее кол-во просмотренных страниц за визит (0.63), проведенное время на сайте за разные периоды (предыдущий месяц — 0.62, текущий месяц — 0.39), среднее количество просматриваемых категорий продуктов за визит (0.48), кол-во неоплаченных продуктов в корзине (0.47), среднемесячная доля покупок по акции от общего числа покупок (0.41);
- **малозначимые**: среднемесячное значение маркетинговых коммуникаций (0.34), выручки за разные периоды (предпредыдущий месяц — 0.18, предыдущий месяц — 0.26, текущий месяц — 0.25), популярная категория (0.17);
- **незначимые**: длительность истории с клиентом (0.09), число сбоев сайта (0.06), кол-во маркетинговых коммуникаций в текущем месяце (0.03), разрешения на сообщения (0.02), тип сервиса (0.01).

Также согласно графику важности SHAP в виде beeswarm, модель говорит о том, что высокие значения среднего кол-ва просмотренных страниц за визит, проведенного времени на сайте за разные периоды, среднего количества просматриваемых категорий продуктов за визит и низкие значения кол-ва неоплаченных продуктов в корзине, среднемесячной доли покупок по акции от общего числа покупок влияют на определение прежнего уровня покупательской активности.

Следовательно, интернет-магазину «В один клик» следует обращать внимание на низкие значения среднего кол-ва просмотренных страниц за визит, проведенного времени на сайте за разные периоды, среднего количества просматриваемых категорий продуктов за визит и высокие значения кол-ва неоплаченных продуктов в корзине, среднемесячной доли покупок по акции от общего числа покупок, так как все они, с точки зрения модели, значительно влияют на определение сниженного уровня покупательской активности.

## Сегментация пользователей

### Объединение данных с результатами модели машинного обучения

Объединение данных с вероятностями принадлежности к сниженному уровню покупательской активности в один датафрейм `new_market`:

In [ ]:
# вычисление вероятностей принадлежности
# к прежнему уровню покупательской активности
y_train_proba = model.named_steps['models'].predict_proba(X_train_prepr)[:,1]
y_test_proba = model.named_steps['models'].predict_proba(X_test_prepr)[:,1]

# объединение
X_train['probability_reduction_buying_activity'] = y_train_proba
X_test['probability_reduction_buying_activity'] = y_test_proba
new_market = pd.concat([X_train, X_test], axis=0)
new_market = new_market.merge(market[['id', 'buying_activity']],
                              on='id', how='inner')
new_market = new_market.merge(money, on='id', how='inner')

# проверка
print(f'Размеры исходных датафреймов: {X_train.shape}, {X_test.shape}, {market[["id", "buying_activity"]].shape} и {money.shape}')
print(f'Размер датафрейма, полученного в результате объединения: {new_market.shape}')

Вывод первых десяти строк нового датафрейма `new_market`:

In [ ]:
new_market.head(10)

**Вывод:**

Было произведено объединение данных с вероятностями принадлежности к сниженному уровню покупательской активности в один датафрейм `new_market` (1285, 20).

### Первый cегмент пользователей

Составление диаграммы рассеяния:

In [ ]:
scatter_plot(df=new_market,
             x='probability_reduction_buying_activity',
             y='pages_per_visit',
             hue='buying_activity',
             xlabel='вероятность снижения покупательской активности',
             ylabel='среднее кол-во просмотренных страниц за визит',
             legend_title='Покуп. активность',
             title='Зависимость между вер. снижения покупательской активн. и средним кол-вом просмотренных страниц за визит')

Выделение сегмента и информация о его прибыли:

In [ ]:
segment1 = new_market[(new_market['probability_reduction_buying_activity'] > 0.6) &\
                      (new_market['pages_per_visit'] < 7.5)]

# вывод информации о прибыли сегмента
segment1['profit'].describe()

**Вывод:**

Согласно диаграмме рассеяния, отображающей зависимость между вероятностью снижения покупательской активности и средним кол-вом просмотренных страниц за визит, можно выделить 368 пользователей, у которых вероятность снижения покупательской активности больше 60% и среднее кол-во просмотренных страниц за визит меньше 7.5. Среднее значение прибыли данного сегмента составляет примерно 4.04.

### Второй cегмент пользователей

Создание среднего времени, проведенного на сайте за предыдущий и текущий месяцы:

In [ ]:
new_market['time'] = (new_market['previous_month_time'] + new_market['current_month_time']) / 2

# проверка
new_market[['previous_month_time', 'current_month_time', 'time']].head(10)

Составление диаграммы рассеяния:

In [ ]:
scatter_plot(df=new_market,
             x='probability_reduction_buying_activity',
             y='time',
             hue='buying_activity',
             xlabel='вероятность снижения покупательской активности',
             ylabel='среднее время, проведенное на сайте',
             legend_title='Покуп. активность',
             title='Зависимость между вер. снижения покупательской активн. и средним временем, проведенным на сайте')

Выделение сегмента и информация о его прибыли:

In [ ]:
segment2 = new_market[(new_market['probability_reduction_buying_activity'] > 0.6) &\
                      (new_market['time'] < 10)]

# вывод информации о прибыли сегмента
segment2['profit'].describe()

**Вывод:**

Согласно диаграмме рассеяния, отображающей зависимость между вероятностью снижения покупательской активности и средним временем, проведенным на сайте, можно выделить 180 пользователей, у которых вероятность снижения покупательской активности больше 60% и среднее время, проведенное на сайте, меньше 10 мин. Среднее значение прибыли данного сегмента составляет примерно 4.02.

### Третий cегмент пользователей

Составление диаграммы рассеяния:

In [ ]:
scatter_plot(df=new_market,
             x='probability_reduction_buying_activity',
             y='average_category_views_per_visit',
             hue='buying_activity',
             xlabel='вероятность снижения покупательской активности',
             ylabel='среднее кол-во просматриваемых категорий продуктов за визит',
             legend_title='Покуп. активность',
             title='Зависимость между вер. снижения покупательской активн. и средним кол-вом просматриваемых категорий продуктов за визит')

Выделение сегмента и информация о его прибыли:

In [ ]:
segment3 = new_market[(new_market['probability_reduction_buying_activity'] > 0.6) &\
                      (new_market['average_category_views_per_visit'] < 3)]

# вывод информации о прибыли сегмента
segment3['profit'].describe()

**Вывод:**

Согласно диаграмме рассеяния, отображающей зависимость между вероятностью снижения покупательской активности и средним кол-вом просматриваемых категорий продуктов за визит можно выделить 222 пользователя, у которых вероятность снижения покупательской активности больше 60% и среднее кол-во просматриваемых категорий продуктов за визит меньше 3. Среднее значение прибыли данного сегмента составляет примерно 4.07.

### Четвертый cегмент пользователей

Составление диаграммы рассеяния:

In [ ]:
scatter_plot(df=new_market,
             x='probability_reduction_buying_activity',
             y='unpaid_products_pieces_quarter',
             hue='buying_activity',
             xlabel='вероятность снижения покупательской активности',
             ylabel='кол-во неоплаченных продуктов в корзине',
             legend_title='Покуп. активность',
             title='Зависимость между вер. снижения покупательской активн. и кол-вом неоплаченных продуктов в корзине')

Выделение сегмента и информация о его прибыли:

In [ ]:
segment4 = new_market[(new_market['probability_reduction_buying_activity'] > 0.6) &\
                      (new_market['unpaid_products_pieces_quarter'] > 4)]

# вывод информации о прибыли сегмента
segment4['profit'].describe()

**Вывод:**

Согласно диаграмме рассеяния, отображающей зависимость между вероятностью снижения покупательской активности и кол-вом неоплаченных продуктов в корзине можно выделить 168 пользователей, у которых вероятность снижения покупательской активности больше 60% и кол-во неоплаченных продуктов в корзине больше 4. Среднее значение прибыли данного сегмента составляет примерно 4.

### Пятый cегмент пользователей

Составление диаграммы рассеяния:

In [ ]:
scatter_plot(df=new_market,
             x='probability_reduction_buying_activity',
             y='promotional_purchases',
             hue='buying_activity',
             xlabel='вероятность снижения покупательской активности',
             ylabel='среднемесячная доля покупок по акции от общего числа покупок',
             legend_title='Покуп. активность',
             title='Зависимость между вер. снижения покупательской активн. и среднемесячной долей покупок по акции от общего числа покупок')

Выделение сегмента и информация о его прибыли:

In [ ]:
segment5 = new_market[(new_market['probability_reduction_buying_activity'] > 0.6) &\
                      (new_market['promotional_purchases'] > 0.6)]

# вывод информации о прибыли сегмента
segment5['profit'].describe()

**Вывод:**

Согласно диаграмме рассеяния, отображающей зависимость между вероятностью снижения покупательской активности и среднемесячной долей покупок по акции от общего числа покупок можно выделить 135 пользователей, у которых вероятность снижения покупательской активности больше 60% и среднемесячная доля покупок по акции от общего числа покупок больше 0.6. Среднее значение прибыли данного сегмента составляет примерно 3.9.

### Графическое и аналитическое исследование 5-го сегмента пользователей

#### Численные признаки

Статистическое описание численных признаков всех пользователей:

In [ ]:
new_market.describe().T

Статистическое описание численных признаков 5-го сегмента пользователей:

In [ ]:
segment5.describe().T

**Вывод:**

У пятого сегмента пользователей наблюдается высокая вероятность снижения покупательской активности, так как среднее значение составляет примерно 0.93, а медиана — 0.97. Также среднее значение прибыли, по сравнению с остальными проблемными сегментами пользователей, значительно снижено и составляет примерно 3.9.

При этом пятый сегмент пользователей заметно отстает от других пользователей в следующих моментах:

- по среднему количеству страниц, которые просмотрел покупатель за один визит на сайт за последние 3 месяца (в 2 раза);
- по выручке за предпредыдущий месяц.

#### Тип сервиса

In [ ]:
piechart(series1=new_market['service_type'], series2=segment5['service_type'],
         title1='все пользователи', title2='5-ый сегмент',
         legend_title='Типы',
         title='Тип сервиса',
         figsize=(16, 7))

**Вывод:**

В пятом сегменте наблюдается больше премиум-пользователей — на 6.6%.

#### Разрешение на сообщения

In [ ]:
piechart(series1=new_market['allow_reporting'], series2=segment5['allow_reporting'],
         title1='все пользователи', title2='5-ый сегмент',
         legend_title='Разрешения',
         title='Разрешение на сообщения',
         figsize=(16, 7))

**Вывод:**

У всех пользователей и пятого сегмента одинаковые соотношения разрешений на сообщения.

#### Наиболее частая категория продукта в заказе

In [ ]:
piechart(series1=new_market['popular_category'], series2=segment5['popular_category'],
         title1='все пользователи', title2='5-ый сегмент',
         legend_title='Категории',
         title='Наиболее частая категория продукта в заказе',
         figsize=(16, 7))

**Вывод:**

В пятом сегменте пользователей, которые интересуются детскими товарами, больше на 7.9%. И также на 4.9% больше пользователей, которые ищут кухонную посуду.

#### Динамика выручки по месяцам

Создание новых датафреймов для построения графиков динамики выручки по месяцам:

In [ ]:
market_revenue = market_money.merge(
    market_file[['id', 'service_type', 'allow_reporting', 'popular_category']],
    on='id',
    how='inner'
)
segment5_revenue = market_revenue[market_revenue['id'].isin(list(segment5['id']))]

# проверка
display(market_revenue.head(5), market_revenue.shape)
display(segment5_revenue.head(5), segment5_revenue.shape)

Построение столбчатых диаграмм динамики выручки по месяцам в группировке по типам сервиса:

In [ ]:
plotbar(df1=market_revenue, x='period', y='revenue', df2=segment5_revenue,
        hue='service_type', order=['предпредыдущий_месяц', 'предыдущий_месяц', 'текущий_месяц'],
        xlabel='период', ylabel='средняя выручка', legend_title='Типы',
        title1='все пользователи', title2='5-ый сегмент',
        title='Динамика выручки по месяцам в группировке по типам сервиса')

**Вывод**

В пятом сегменте за предыдущий и текущий месяцы меньше средней выручки от стандартных пользователей.

Построение столбчатых диаграмм динамики выручки по месяцам в группировке по разрешению на сообщения:

In [ ]:
plotbar(df1=market_revenue, x='period', y='revenue', df2=segment5_revenue,
        hue='allow_reporting', order=['предпредыдущий_месяц', 'предыдущий_месяц', 'текущий_месяц'],
        xlabel='период', ylabel='средняя выручка', legend_title='Разрешения',
        title1='все пользователи', title2='5-ый сегмент',
        title='Динамика выручки по месяцам в группировке по разрешению на сообщения')

**Вывод**

В пятом сегменте за предыдущий и текущий месяцы меньше средней выручки от пользователей с разрешением на сообщения.

Построение столбчатых диаграмм динамики выручки по месяцам в группировке по популярным категориям:

In [ ]:
plotbar(df1=market_revenue, x='period', y='revenue',
        hue='popular_category', order=['предпредыдущий_месяц', 'предыдущий_месяц', 'текущий_месяц'],
        xlabel='период', ylabel='средняя выручка', legend_title='Категории',
        title1='все пользователи', title2='5-ый сегмент',
        title='Динамика выручки по месяцам в группировке по популярным категориям для всех пользователей')

plotbar(df1=segment5_revenue, x='period', y='revenue',
        hue='popular_category', order=['предпредыдущий_месяц', 'предыдущий_месяц', 'текущий_месяц'],
        xlabel='период', ylabel='средняя выручка', legend_title='Категории',
        title1='все пользователи', title2='5-ый сегмент',
        title='Динамика выручки по месяцам в группировке по популярным категориям для 5-го сегмента')

**Вывод:**

В пятом сегменте за предыдущий и текущий месяцы меньше средней выручки от продажи товаров для красоты и здоровья, чем от других товаров.

### Вывод сегментации пользователей

#### Объединение данных с результатами модели машинного обучения

Было произведено объединение данных с вероятностями принадлежности к сниженному уровню покупательской активности в один датафрейм `new_market` (1285, 20).

#### Первый cегмент пользователей

Согласно диаграмме рассеяния, отображающей зависимость между вероятностью снижения покупательской активности и средним кол-вом просмотренных страниц за визит, можно выделить 368 пользователей, у которых вероятность снижения покупательской активности больше 60% и среднее кол-во просмотренных страниц за визит меньше 7.5. Среднее значение прибыли данного сегмента составляет примерно 4.04.

#### Второй cегмент пользователей

Согласно диаграмме рассеяния, отображающей зависимость между вероятностью снижения покупательской активности и средним временем, проведенным на сайте, можно выделить 180 пользователей, у которых вероятность снижения покупательской активности больше 60% и среднее время, проведенное на сайте, меньше 10 мин. Среднее значение прибыли данного сегмента составляет примерно 4.02.

#### Третий cегмент пользователей

Согласно диаграмме рассеяния, отображающей зависимость между вероятностью снижения покупательской активности и средним кол-вом просматриваемых категорий продуктов за визит можно выделить 222 пользователя, у которых вероятность снижения покупательской активности больше 60% и среднее кол-во просматриваемых категорий продуктов за визит меньше 3. Среднее значение прибыли данного сегмента составляет примерно 4.07.

#### Четвертый cегмент пользователей

Согласно диаграмме рассеяния, отображающей зависимость между вероятностью снижения покупательской активности и кол-вом неоплаченных продуктов в корзине можно выделить 168 пользователей, у которых вероятность снижения покупательской активности больше 60% и кол-во неоплаченных продуктов в корзине больше 4. Среднее значение прибыли данного сегмента составляет примерно 4.

#### Пятый cегмент пользователей

Согласно диаграмме рассеяния, отображающей зависимость между вероятностью снижения покупательской активности и среднемесячной долей покупок по акции от общего числа покупок можно выделить 135 пользователей, у которых вероятность снижения покупательской активности больше 60% и среднемесячная доля покупок по акции от общего числа покупок больше 0.6. Среднее значение прибыли данного сегмента составляет примерно 3.9.

#### Графическое и аналитическое исследование 5-го сегмента пользователей

У пятого сегмента пользователей наблюдается высокая вероятность снижения покупательской активности, так как среднее значение составляет примерно 0.93, а медиана — 0.97. Также среднее значение прибыли, по сравнению с остальными проблемными сегментами пользователей, значительно снижено и составляет примерно 3.9.

При этом пятый сегмент пользователей заметно отстает от других пользователей в следующих моментах:

- по среднему количеству страниц, которые просмотрел покупатель за один визит на сайт за последние 3 месяца (в 2 раза);
- по выручке за предпредыдущий месяц.

В пятом сегменте наблюдается больше премиум-пользователей — на 6.6%.

У всех пользователей и пятого сегмента одинаковые соотношения разрешений на сообщения.

В пятом сегменте пользователей, которые интересуются детскими товарами, больше на 7.9%. И также на 4.9% больше пользователей, которые ищут кухонную посуду.

В пятом сегменте за предыдущий и текущий месяцы меньше средней выручки от стандартных пользователей.

В пятом сегменте за предыдущий и текущий месяцы меньше средней выручки от пользователей с разрешением на сообщения.

В пятом сегменте за предыдущий и текущий месяцы меньше средней выручки от продажи товаров для красоты и здоровья, чем от других товаров.

## Общий вывод

### Загрузка данных

#### Датасет `market_file.csv`

Создан датафрейм `market_file`, содержащий в себе данные о поведении покупателя на сайте, о коммуникациях с покупателем и его продуктовом поведении. Он имеет 13 столбцов и 1300 строк.

Столбцы имеют следующий характер:

- `id` — номер покупателя в корпоративной базе данных. Имеет тип `int64`, а значения распределены от 215348 до 216647.
- `Покупательская активность` — рассчитанный класс покупательской активности (целевой признак): `снизилась` или `прежний уровень`. Имеет тип `object` и 2 уникальных значения.
- `Тип сервиса` — уровень сервиса, например `премиум` и `стандарт`. Имеет тип `object` и 3 уникальных значения.
- `Разрешить сообщать` — информация о том, можно ли присылать покупателю дополнительные предложения о товаре. Согласие на это даёт покупатель. Имеет тип `object` и 2 уникальных значения.
- `Маркет_актив_6_мес` — среднемесячное значение маркетинговых коммуникаций компании, которое приходилось на покупателя за последние 6 месяцев. Это значение показывает, какое число рассылок, звонков, показов рекламы и прочего приходилось на клиента. Имеет тип `float64`, а значения распределены от 0.9 до 6.6, при этом среднее — примерно 4.25.
- `Маркет_актив_тек_мес` — количество маркетинговых коммуникаций в текущем месяце. Имеет тип `int64`, а значения распределены от 3 до 5, при этом среднее — примерно 4.
- `Длительность` — значение, которое показывает, сколько дней прошло с момента регистрации покупателя на сайте. Имеет тип `int64`, а значения распределены от 110 до 1079 дней, при этом среднее — примерно 602 дня.
- `Акционные_покупки` — среднемесячная доля покупок по акции от общего числа покупок за последние 6 месяцев. Имеет тип `float64`, а значения распределены от 0 до 0.99, при этом среднее — примерно 0.32.
- `Популярная_категория` — самая популярная категория товаров у покупателя за последние 6 месяцев. Имеет тип `object` и 6 уникальных значений.
- `Средний_просмотр_категорий_за_визит` — показывает, сколько в среднем категорий покупатель просмотрел за визит в течение последнего месяца. Имеет тип `int64`, а значения распределены от 1 до 6 категорий, при этом среднее — примерно 3 категории.
- `Неоплаченные_продукты_штук_квартал` — общее число неоплаченных товаров в корзине за последние 3 месяца. Имеет тип `int64`, а значения распределены от 0 до 10 товаров, при этом среднее — примерно 3 товара.
- `Ошибка_сервиса` — число сбоев, которые коснулись покупателя во время посещения сайта. Имеет тип `int64`, а значения распределены от 0 до 9 сбоев, при этом среднее — примерно 4 сбоя.
- `Страниц_за_визит` — среднее количество страниц, которые просмотрел покупатель за один визит на сайт за последние 3 месяца. Имеет тип `int64`, а значения распределены от 1 до 20 страниц, при этом среднее — примерно 8 страниц.

#### Датасет `market_money.csv`

Создан датафрейм `market_money`, содержащий в себе данные о выручке, которую получает магазин с покупателя (то есть сколько покупатель всего потратил за период взаимодействия с сайтом). Он имеет 3 столбца и 3900 строк.

Столбцы:

- `id` — номер покупателя в корпоративной базе данных. Имеет тип `int64`, а значения распределены от 215348 до 216647.
- `Период` — название периода, во время которого зафиксирована выручка. Например, `текущий_месяц` или `предыдущий_месяц`. Имеет тип `object` и 3 уникальных значения.
- `Выручка` — сумма выручки за период. Имеет тип `float64`, а значения распределены от 0 до 106862.2, при этом среднее — примерно 5025.7.

#### Датасет `market_time.csv`

Создан датафрейм `market_time`, содержащий в себе данные о времени (в минутах), которое покупатель провёл на сайте в течение периода. Он имеет 3 столбца и 2600 строк.

Столбцы:

- `id` — номер покупателя в корпоративной базе данных. Имеет тип `int64`, а значения распределены от 215348 до 216647.
- `Период` — название периода, во время которого зафиксировано общее время. Имеет тип `object` и 2 уникальных значения.
- `минут` — значение времени, проведённого на сайте, в минутах. Имеет тип `int64`, а значения распределены от 4 до 23 мин, при этом среднее — примерно 13 мин.

#### Датасет `money.csv`

Создан датафрейм `money`, содержащий в себе данные о среднемесячной прибыли покупателя за последние 3 месяца (какую прибыль получает магазин от продаж каждому покупателю). Он имеет 2 столбца и 1300 строк.

Столбцы:

- `id` — номер покупателя в корпоративной базе данных. Имеет тип `int64`, а значения распределены от 215348 до 216647.
- `Прибыль` — значение прибыли. Имеет тип `float64`, а значения распределены от 0.86 до 7.43, при этом среднее — примерно 4.

### Предобработка данных

#### Переименование столбцов

Названия столбцов датафрейма `market_file` переименованы согласно PEP8. Новые названия столбцов:

- `id` — номер покупателя в корпоративной базе данных;
- `buying_activity` — рассчитанный класс покупательской активности;
- `service_type` — уровень сервиса;
- `allow_reporting` — информация о том, можно ли присылать покупателю дополнительные предложения о товаре;
- `market_active_6_months` — среднемесячное значение маркетинговых коммуникаций компании, которое приходилось на покупателя за последние 6 месяцев;
- `market_active_tech_mes` — количество маркетинговых коммуникаций в текущем месяце;
- `duration` — значение, которое показывает, сколько дней прошло с момента регистрации покупателя на сайте;
- `promotional_purchases` — среднемесячная доля покупок по акции от общего числа покупок за последние 6 месяцев;
- `popular_category` — самая популярная категория товаров у покупателя за последние 6 месяцев;
- `average_category_views_per_visit` — показывает, сколько в среднем категорий покупатель просмотрел за визит в течение последнего месяца;
- `unpaid_products_pieces_quarter` — общее число неоплаченных товаров в корзине за последние 3 месяца;
- `service_error` — число сбоев, которые коснулись покупателя во время посещения сайта;
- `pages_per_visit` — среднее количество страниц, которые просмотрел покупатель за один визит на сайт за последние 3 месяца.

Названия столбцов датафрейма `market_money` переименованы согласно PEP8. Новые названия столбцов:

- `id` — номер покупателя в корпоративной базе данных;
- `period` — название периода, во время которого зафиксирована выручка;
- `revenue` — сумма выручки за период.

Названия столбцов датафрейма `market_time` переименованы согласно PEP8. Новые названия столбцов:

- `id` — номер покупателя в корпоративной базе данных;
- `period` — название периода, во время которого зафиксировано общее время;
- `minutes` — значение времени, проведённого на сайте, в минутах.

Названия столбцов датафрейма `money` переименованы согласно PEP8. Новые названия столбцов:

- `id` — номер покупателя в корпоративной базе данных;
- `profit` — значение прибыли.

#### Проверка типов данных

В датафрейме `market_file` все столбцы имеют корректные типы данных. Однако значения столбцов `market_active_6_months`, `promotional_purchases` были переведены из `float64` в `float32` для экономии вычислительной памяти.

В датафрейме `market_money` все столбцы имеют корректные типы данных. Однако значения столбца `revenue` были переведены из `float64` в `float32` для экономии вычислительной памяти.

В датафрейме `market_time` все столбцы имеют корректные типы данных.

В датафрейме `money` все столбцы имеют корректные типы данных. Однако значения столбца `profit` были переведены из `float64` в `float32` для экономии вычислительной памяти.

#### Изучение пропущенных значений

В датафреймах `market_file`, `market_money`, `market_time` и `money` пропущенных значений не наблюдается.

#### Изучение дубликатов

В столбце `service_type` датафрейма `market_file` были выявлены неявные дубликаты, поэтому их строковые значения были заменены следующим образом: `стандартт` на `стандарт`. Также в столбце `popular_category` исправлена опечатка: `Косметика и аксесуары` на `Косметика и аксессуары`. Еще значения в столбцах `buying_activity` и `popular_category` были приведены к нижнему регистру. При этом было выявлено и удалено 11 явных дубликата по всем столбцам кроме `id`.

В столбце `period` датафрейма `market_money` была найдена опечатка, поэтому строковые значения были заменены следующим образом: `препредыдущий_месяц` на `предпредыдущий_месяц`. При этом неявных и явных дубликатов не обнаружено.

В столбце `period` датафрейма `market_time` была найдена опечатка, поэтому строковые значения были заменены следующим образом: `предыдцщий_месяц` на `предыдущий_месяц`. При этом неявных и явных дубликатов не обнаружено.

В датафрейме `money` неявных и явных дубликатов не обнаружено.

### Исследовательский анализ данных

#### Коммуникация с пользователем

Большинство пользователей предпочитают стандартный тип сервиса как при снижении покупательской активности (65.3%, 318 пользователей), так и при её сохранении на прежнем уровне (74.3%, 596 пользователей). Причем снижение покупательской активности приводит к увеличению доли клиентов (на 9%), использующих премиум-сервис.

Большинство пользователей предпочитают включать разрешения на сообщения как при снижении покупательской активности (74.7%, 364 пользователя), так и при её сохранении на прежнем уровне (73.7%, 591 пользователь).

Распределение маркетинга за 6 месяцев более симметрично у пользователей со сниженной покупательской активностью. Обе группы в большинстве своем имеют среднемесячное значение маркетинговых коммуникаций примерно равное 4. При этом у обоих распределений максимальное значение составляет 6.6, а минимальное — 0.9. В предоставленных данных, пользователи с прежним уровнем покупательской активности имеют большее среднемесячное значение маркетинговых коммуникаций (медиана — 4.4) по сравнению с пользователями, у которых активность снизилась (медиана — 3.9). При этом межквартильные размахи у обоих групп пользователей практически совпадают (у сниженного уровня — 1.3, а у прежнего — 1.27), что указывает на примерно одинаковое распределение данных. Также в сниженном уровне покупательской активности наблюдается 2.26% выбросов (в значениях меньше 1 или больше 6), а в прежнем — 0.25% (в значениях меньше 2).

Распределение маркетинга за текущий месяц более симметрично у пользователей со сниженной покупательской активностью. Обе группы в большинстве своем имеют количество маркетинговых коммуникаций равное 4. При этом у обоих распределений максимальное значение составляет 5, а минимальное — 3. В предоставленных данных, пользователи обоих групп имеют одинаковое кол-во маркетинговых коммуникаций (медиана — 4). Межквартильный размах у пользователей с прежним уровнем покупательской активности слишком мал (0), и по этой причине отображается огромная доля выбросов (47.5%). У сниженного же уровня покупательской активности никаких выбросов не наблюдается, а межквартильный размах находится в диапазоне от 4 до 5 маркетинговых коммуникаций.

Распределение длительности истории является многомодальным у обоих групп пользователей и имеет пики примерно в 500 и 800 дней. У пользователей со сниженной покупательской активностью минимальное значение составляет 110 дней, а максимальное — 1079 дней. С прежним уровнем — 121 и 1061 дней соответсвтенно. В предоставленных данных, пользователи со сниженным уровнем покупательской активности зарегистрированы на сайте дольше (медиана — 637 дней) по сравнению с пользователями, у которых активность сохранилась на прежнем уровне (медиана — 590 дней). При этом межквартильный размах для пользователей с прежним уровнем покупательской активности шире (на 54 дня), что указывает на более широкое распределение данных. Выбросов в обоих группах не обнаружено.

#### Продуктовое поведение

Товары для детей являются наиболее частой категорией продуктов в заказах для обеих групп пользователей, но их доля выше (на 6.5%) среди пользователей с пониженной покупательской активностью. Косметика, аксессуары и домашний текстиль также популярны среди пользователей с пониженной активностью, но их доли ниже среди пользователей с прежним уровнем покупательской активности (косметика и аксессуары — на 6.7%, домашний текстиль — на 1.9%). Мелкая бытовая техника и электроника значительно более популярна среди пользователей с прежним уровнем покупательской активности по сравнению с пользователями с пониженной (на 13.2%).

Распределение среднего количества просматриваемых категорий продуктов за визит более симметрично у пользователей с прежним уровнем покупательской активности, при этом в большинстве своем они имеют среднее кол-во просматриваемых категорий равное 3 и 4. У сниженного же уровня покупательской активности наиболее частое значение — 2, что значительно меньше. Также у обоих распределений максимальное значение составляет 6, а минимальное — 1. В предоставленных данных, пользователи с прежним уровнем покупательской активности просматривают в 2 раза больше категорий продуктов за визит (медиана — 4) по сравнению с пользователями, у которых покупательская активность снизилась (медиана — 2). При этом межквартильный размах для пользователей с прежним уровнем покупательской активности шире (на 1), что указывает на более широкое распределение данных. Также в сниженном уровне покупательской активности наблюдается 8.42% выбросов (в значениях больше или равно 5), а в прежнем — они отсутствуют.

Распределение количества неоплаченных продуктов в корзине за последние 3 месяца более симметрично у пользователей со сниженным уровнем покупательской активности, при этом в большинстве своем они имеют кол-во неоплаченных продуктов в размере от 2 до 5. У прежнего же уровня покупательской активности наиболее частое значение — 1, что значительно меньше. Также у обоих распределений минимальные значения совпадают (в значении 0), а максимальные расходятся (у сниженного уровня — 10, а у прежнего — 8). В предоставленных данных, пользователи с прежним уровнем покупательской активности имеют в 2 раза меньше неоплаченных продуктов в корзине (медиана — 2) по сравнению с пользователями, у которых покупательская активность снизилась (медиана — 4). При этом межквартильный размах для пользователей со сниженным уровнем покупательской активности шире (на 1), что указывает на более широкое распределение данных. Также в сниженном уровне покупательской активности наблюдается 0.82% выбросов (в значениях больше или равно 10), а в прежнем — 0.37% (в значениях больше или равно 7).

#### Финансовое поведение

Распределение разницы в выручке предыдущего и предпредыдущего месяца более симметрично у пользователей со сниженным уровнем покупательской активности, при этом в большинстве своем они имеют разницу в выручке в размере примерно 428. У прежнего же уровня покупательской активности наиболее частое значение — примерно 214, что значительно меньше. Также у обоих распределений минимальные значения совпадают 0, а максимальные расходятся (у сниженного уровня — 2067, а у прежнего — 1999). В предоставленных данных, пользователи с прежним уровнем покупательской активности имеют меньшую разницу в выручке (медиана — 392.5) по сравнению с пользователями, у которых покупательская активность снизилась (медиана — 657). При этом межквартильный размах для пользователей со сниженным уровнем покупательской активности шире (на 145.63), что указывает на более широкое распределение данных. Также в сниженном уровне покупательской активности наблюдается 0.2% выбросов (в значениях больше 2000), а в прежнем — 0.37% (в значениях больше или равно 1500).

Распределение разницы в выручке текущего и предыдущего месяца более симметрично у пользователей со сниженным уровнем покупательской активности, при этом в большинстве своем они имеют разницу в выручке в размере примерно 250. У прежнего же уровня покупательской активности наиболее частое значение — примерно 167, что значительно меньше. Также у обоих распределений минимальные (у сниженного уровня — 1.3, а у прежнего — 0) и максимальные (у сниженного уровня — 5986.3, а у прежнего — 1446.9) значения не совпадают. В предоставленных данных, обе группы пользователей имеют примерно одинаковые разницы в выручке текущего и предыдущего месяца (медиана у сниженного уровня — 388.7, а у прежнего — 373.1). При этом межквартильный размах для пользователей с прежним уровнем покупательской активности шире (на 57.2), что указывает на более широкое распределение данных. Также в сниженном уровне покупательской активности было удалено одно аномальное значение (100785.2), причем после удаления наблюдается 1.65% выбросов (в значениях больше 1300). В прежнем же уровне — 0.25% выбросов (в значениях больше 1400).

Распределение выручки за текущий месяц более симметрично у пользователей с прежним уровнем покупательской активности, при этом в большинстве своем они имеют выручку в размере примерно 5000. У сниженного же уровня покупательской активности наиболее частое значение — примерно 5333. Также у обоих распределений минимальные (у сниженного уровня — 2758.7, а у прежнего — 2952.2) и максимальные (у сниженного уровня — 7799.4, а у прежнего — 7547.8) значения не совпадают. В предоставленных данных, пользователи с прежним уровнем покупательской активности имеют меньшую выручку (медиана — 5122.55) по сравнению с пользователями, у которых покупательская активность снизилась (медиана — 5296.7). При этом межквартильный размах для пользователей со сниженным уровнем покупательской активности шире (на 436.25), что указывает на более широкое распределение данных. Также в сниженном уровне покупательской активности выбросов не обнаружилось, а в прежнем — 4.11% выбросов (в значениях меньше 3500 или больше 6800).

#### Поведение на сайте

Распределение времени, проведенного на сайте за предыдущий месяц, более равномерно у пользователей со сниженным уровнем покупательской активности, при этом в большинстве своем они имеют время примерно равное 10 мин. У прежнего же уровня покупательской активности наиболее частое значение — примерно 15 мин. Также у обоих распределений минимальные значения не совпадают (у сниженного уровня — 5 мин, а у прежнего — 7 мин), а максимальные наоборот — совпадают (23 мин). В предоставленных данных, пользователи с прежним уровнем покупательской активности в прошлом месяце провели больше времени за сайтом (медиана — 15 мин) по сравнению с пользователями, у которых покупательская активность снизилась (медиана — 10 мин). При этом межквартильные размахи пользователей обоих групп совпадают (5 мин), что указывает на примерно одинаковое распределение данных. Также в прежнем уровне покупательской активности выбросов не обнаружилось, а в сниженном — 1.44% выбросов (в значениях больше 20 мин).

Распределение времени, проведенного на сайте за текущий месяц, более равномерно у пользователей со сниженным уровнем покупательской активности, при этом в большинстве своем они имеют время примерно равное от 8 до 11 мин. У прежнего же уровня покупательской активности наиболее частое значение — примерно 15 мин. Также у обоих распределений минимальные значения совпадают (4 мин), а максимальные наоборот — не совпадают (у сниженного уровня — 22 мин, а у прежнего — 23 мин). В предоставленных данных, пользователи с прежним уровнем покупательской активности в текущем месяце провели больше времени за сайтом (медиана — 15 мин) по сравнению с пользователями, у которых покупательская активность снизилась (медиана — 10 мин). При этом межквартильный размах для пользователей с прежним уровнем покупательской активности шире (на 2 мин), что указывает на более широкое распределение данных. Также в прежнем уровне покупательской активности выбросов не обнаружилось, а в сниженном — 2.46% выбросов (в значениях больше или равно 20 мин).

Распределение количества просматриваемых страниц за визит более симметрично у пользователей с прежним уровнем покупательской активности, при этом в большинстве своем они просматривают 10 страниц. У сниженного же уровня покупательской активности наиболее частое значение — 3-4 страницы, что значительно ниже. Также у обоих распределений минимальные (у сниженного уровня — 1 стр., а у прежнего — 3 стр.) и максимальные (у сниженного уровня — 18 стр., а у прежнего — 20 стр.) значения не совпадают. В предоставленных данных, пользователи с прежним уровнем покупательской активности просматривают в 2 раза больше страниц (медиана — 10 стр.) по сравнению с пользователями, у которых покупательская активность снизилась (медиана — 5 стр.). При этом межквартильный размах для пользователей с прежним уровнем покупательской активности шире (на 1 стр.), что указывает на более широкое распределение данных. Также в сниженном уровне покупательской активности наблюдается 3.9% выбросов (в значениях больше или равно 14 стр.), а в прежнем — 0.25% (в значениях больше или равно 20 стр.).

Распределение количества сбоев сайта более симметрично у пользователей с прежним уровнем покупательской активности, при этом в большинстве своем они имеют 4 сбоя. У сниженного же уровня покупательской активности наиболее частое значение — 3 сбоя. Также у обоих распределений минимальные значения не совпадают (у сниженного уровня — 1 сбой, а у прежнего — 0 сбоев), а максимальные наоборот — совпадают (9 сбоев). В предоставленных данных, пользователи обоих групп имеют одинаковое количество сбоев сайта (медиана — 4 сбоя). И их межквартильные размахи тоже совпадают (3 сбоя), что указывает на примерно одинаковое распределение данных. Также выбросов в обоих группах не замечено.

#### Отбор пользователей с покупательской активностью не менее трех месяцев

Во всех датафреймах были удалены пользователи (`id`: 215348, 215357, 215359), которые за последние 3 месяца хотя бы в одном месяце не принесли выручку.

#### Покупательская активность

В значениях покупательской активности наблюдается небольшой дисбаланс. Доля пользователей с прежним уровнем покупательской активности составляет 62.4% (802 пользователя), а со сниженным уровнем — 37.6% (484 пользователя). В свою очередь это может отразиться на обучении модели машинного обучения и её метриках качества.

### Объединение данных

Было произведено объединение датафреймов `market_file` (1286, 13), `market_money_pivot` (1296, 4) и `market_time_pivot` (1297, 3) в один датафрейм `market` (1285, 18).

### Корреляционный анализ

#### Анализ с помощью коэффициента $\phi_k$

Согласно коэффициенту $\phi_k$, в датафрейме `market` наблюдаются следующие сильные корреляции:

- между целевым признаком `buying_activity` и: `pages_per_visit` (0.8), `previous_month_time` (0.7);
- между входными признаками: `current_month_revenue` и `previous_month_revenue` (0.8).

Очень высоких корреляций (> 0.9) между входными признаками не наблюдается, скорее всего мультиколлинеарности нет.

Также никаких сильных новых корреляций для каждой категории целевого признака `buying_activity` не выявлено.

#### Анализ с помощью диаграмм рассеяния

Наблюдается явная линейная зависимость между выручками за текущий и прошлый месяцы. Также замечена довольно значительная часть пользователей, которые за последние полгода большую часть покупок совершали по акции, причем у средних `id` отсутствует такое поведение.

Наблюдается сильная линейная зависимость между выручками у пользователей со сниженной покупательской активностью, с премиум-сервисом и с популярной категорией "товары для детей".

Большинство пользователей, которые за последние полгода чаще всего совершали покупки по акциям, отличаются сниженной покупательской активностью, используют стандартный сервис и разрешают получать сообщения.

### Построение модели машинного обучения

#### Выбор признаков

В качестве целевого признака был выбран столбец `buying_activity`, а в качестве входных — остальные столбцы из датафрейма `market`, кроме `id`. Также было произведено кодирование (label encoder) целевого признака (прежний уровень покупательской активности обозначается как `0`, а сниженный — как `1`). Затем данные были разделены на тренировочную (75%) и тестовую выборки (25%) с применением стратификации.

#### Пайплайны

Для подготовки данных были выбраны следующие методы:

- кодирование: `OneHotEncoder()` и `OrdinalEncoder()`;
- масштабирование: `StandardScaler()`, `MinMaxScaler()` и `RobustScaler()`.

Для задачи классификации было выбрано 4 модели машинного обучения:

- `KNeighborsClassifier()` — метод k-ближайших соседей с гиперпараметрами: кол-во соседей от 2 до 10 и манхеттанское, евклидово расстояния;
- `DecisionTreeClassifier()` — дерево решений с гиперпараметрами: максимальная глубина дерева от 2 до 10, минимальное кол-во объектов в узле от 2 до 10 и минимальное кол-во объектов в листе от 1 до 10;
- `SVC()` — метод опорных векторов с гиперпараметрами: сила регуляризации от 1 до 10 и линейное, радиальная базисная функция, сигмоидное, полиномиальное ядра;
- `LogisticRegression()` — логистическая регрессия с l1-регуляризацией с гиперпараметром сила регуляризации от 1 до 10.

#### Кросс-валидация

Для оценки качества модели была выбрана метрика `ROC-AUC`, так как она не зависит от порога классификации, учитывает как `True Positive Rate`, так и `False Positive Rate`, что делает её более устойчивой к дисбалансу классов по сравнению с метриками, такими как `accuracy`.

При случайной кросс-валидации лучшей моделью оказалось логистическая регрессия с l1-регуляризацией и гиперпараметром `C` равным 9. Причем категориальные данные подготавливались с помощью кодировщиков `OneHotEncoding()` (`service_type` и `allow_reporting`) и `OrdinalEncoder()` (`popular_category`), а численные остались неизменными. Значение `ROC-AUC` лучшей модели на валидационных данных составляет 0.9, а на тестовых — 0.91.

#### Анализ модели

Матрица ошибок лучшей модели имеет следующий вид:

- 190 пользователей с прежним уровнем покупательской активности были верно классифицированы (TN),
- 22 пользователя со сниженным уровнем покупательской активности были неверно классифицированы (FN),
- 99 пользователей со сниженным уровнем покупательской активности были верно классифицированы (TP),
- 11 пользователей с прежним уровнем покупательской активности был неверно классифицирован (FP).

Доля верных прогнозов на тестовой выборке составляет 90% (accuracy), причем стоит учесть, что в предоставленных данных пользователей с прежним уровнем покупательской активности на 24.8% больше, чем со сниженным уровнем. Точность же, с которой модель присваивает пользователям категорию `снизилась` составляет 90% (precision). Причем 82% (recall) пользователей категории `прежний уровень` модель смогла правильно идентифицировать среди всех пользователей категории `прежний уровень`.

#### Графики важности SHAP

С точки зрения модели и согласно графику важности SHAP в виде плотбара, входные признаки можно разделить на:

- **значимые**: среднее кол-во просмотренных страниц за визит (0.63), проведенное время на сайте за разные периоды (предыдущий месяц — 0.62, текущий месяц — 0.39), среднее количество просматриваемых категорий продуктов за визит (0.48), кол-во неоплаченных продуктов в корзине (0.47), среднемесячная доля покупок по акции от общего числа покупок (0.41);
- **малозначимые**: среднемесячное значение маркетинговых коммуникаций (0.34), выручки за разные периоды (предпредыдущий месяц — 0.18, предыдущий месяц — 0.26, текущий месяц — 0.25), популярная категория (0.17);
- **незначимые**: длительность истории с клиентом (0.09), число сбоев сайта (0.06), кол-во маркетинговых коммуникаций в текущем месяце (0.03), разрешения на сообщения (0.02), тип сервиса (0.01).

Также согласно графику важности SHAP в виде beeswarm, модель говорит о том, что высокие значения среднего кол-ва просмотренных страниц за визит, проведенного времени на сайте за разные периоды, среднего количества просматриваемых категорий продуктов за визит и низкие значения кол-ва неоплаченных продуктов в корзине, среднемесячной доли покупок по акции от общего числа покупок влияют на определение прежнего уровня покупательской активности.

Следовательно, интернет-магазину «В один клик» следует обращать внимание на низкие значения среднего кол-ва просмотренных страниц за визит, проведенного времени на сайте за разные периоды, среднего количества просматриваемых категорий продуктов за визит и высокие значения кол-ва неоплаченных продуктов в корзине, среднемесячной доли покупок по акции от общего числа покупок, так как все они, с точки зрения модели, значительно влияют на определение сниженного уровня покупательской активности.

### Сегментация пользователей

#### Объединение данных с результатами модели машинного обучения

Было произведено объединение данных с вероятностями принадлежности к сниженному уровню покупательской активности в один датафрейм `new_market` (1285, 20).

#### Первый cегмент пользователей

Согласно диаграмме рассеяния, отображающей зависимость между вероятностью снижения покупательской активности и средним кол-вом просмотренных страниц за визит, можно выделить 368 пользователей, у которых вероятность снижения покупательской активности больше 60% и среднее кол-во просмотренных страниц за визит меньше 7.5. Среднее значение прибыли данного сегмента составляет примерно 4.04.

#### Второй cегмент пользователей

Согласно диаграмме рассеяния, отображающей зависимость между вероятностью снижения покупательской активности и средним временем, проведенным на сайте, можно выделить 180 пользователей, у которых вероятность снижения покупательской активности больше 60% и среднее время, проведенное на сайте, меньше 10 мин. Среднее значение прибыли данного сегмента составляет примерно 4.02.

#### Третий cегмент пользователей

Согласно диаграмме рассеяния, отображающей зависимость между вероятностью снижения покупательской активности и средним кол-вом просматриваемых категорий продуктов за визит можно выделить 222 пользователя, у которых вероятность снижения покупательской активности больше 60% и среднее кол-во просматриваемых категорий продуктов за визит меньше 3. Среднее значение прибыли данного сегмента составляет примерно 4.07.

#### Четвертый cегмент пользователей

Согласно диаграмме рассеяния, отображающей зависимость между вероятностью снижения покупательской активности и кол-вом неоплаченных продуктов в корзине можно выделить 168 пользователей, у которых вероятность снижения покупательской активности больше 60% и кол-во неоплаченных продуктов в корзине больше 4. Среднее значение прибыли данного сегмента составляет примерно 4.

#### Пятый cегмент пользователей

Согласно диаграмме рассеяния, отображающей зависимость между вероятностью снижения покупательской активности и среднемесячной долей покупок по акции от общего числа покупок можно выделить 135 пользователей, у которых вероятность снижения покупательской активности больше 60% и среднемесячная доля покупок по акции от общего числа покупок больше 0.6. Среднее значение прибыли данного сегмента составляет примерно 3.9.

#### Графическое и аналитическое исследование 5-го сегмента пользователей

У пятого сегмента пользователей наблюдается высокая вероятность снижения покупательской активности, так как среднее значение составляет примерно 0.93, а медиана — 0.97. Также среднее значение прибыли, по сравнению с остальными проблемными сегментами пользователей, значительно снижено и составляет примерно 3.9.

При этом пятый сегмент пользователей заметно отстает от других пользователей в следующих моментах:

- по среднему количеству страниц, которые просмотрел покупатель за один визит на сайт за последние 3 месяца (в 2 раза);
- по выручке за предпредыдущий месяц.

В пятом сегменте наблюдается больше премиум-пользователей — на 6.6%.

У всех пользователей и пятого сегмента одинаковые соотношения разрешений на сообщения.

В пятом сегменте пользователей, которые интересуются детскими товарами, больше на 7.9%. И также на 4.9% больше пользователей, которые ищут кухонную посуду.

В пятом сегменте за предыдущий и текущий месяцы меньше средней выручки от стандартных пользователей.

В пятом сегменте за предыдущий и текущий месяцы меньше средней выручки от пользователей с разрешением на сообщения.

В пятом сегменте за предыдущий и текущий месяцы меньше средней выручки от продажи товаров для красоты и здоровья, чем от других товаров.